In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Sentiment Analysis Web App\n",
    "\n",
    "_Deep Learning Nanodegree Program | Deployment_\n",
    "\n",
    "---\n",
    "\n",
    "In this notebook we will use Amazon's SageMaker service to construct a random tree model to predict the sentiment of a movie review. In addition, we will deploy this model to an endpoint and construct a very simple web app which will interact with our model's deployed endpoint.\n",
    "\n",
    "## General Outline\n",
    "\n",
    "Typically, when using a notebook instance with SageMaker, you will proceed through the following steps. Of course, not every step will need to be done with each project. Also, there is quite a lot of room for variation in many of the steps, as you will see throughout these lessons.\n",
    "\n",
    "1. Download or otherwise retrieve the data.\n",
    "2. Process / Prepare the data.\n",
    "3. Upload the processed data to S3.\n",
    "4. Train a chosen model.\n",
    "5. Test the trained model (typically using a batch transform job).\n",
    "6. Deploy the trained model.\n",
    "7. Use the deployed model.\n",
    "\n",
    "In this notebook we will progress through each of the steps above. We will also see that the final step, using the deployed model, can be quite challenging."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 1: Downloading the data\n",
    "\n",
    "The dataset we are going to use is very popular among researchers in Natural Language Processing, usually referred to as the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/). It consists of movie reviews from the website [imdb.com](http://www.imdb.com/), each labeled as either '**pos**itive', if the reviewer enjoyed the film, or '**neg**ative' otherwise.\n",
    "\n",
    "> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.\n",
    "\n",
    "We begin by using some Jupyter Notebook magic to download and extract the dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "mkdir: cannot create directory ‘../data’: File exists\n",
      "--2019-01-14 18:20:03--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz\n",
      "Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10\n",
      "Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.\n",
      "HTTP request sent, awaiting response... 200 OK\n",
      "Length: 84125825 (80M) [application/x-gzip]\n",
      "Saving to: ‘../data/aclImdb_v1.tar.gz’\n",
      "\n",
      "../data/aclImdb_v1. 100%[===================>]  80.23M  10.1MB/s    in 9.9s    \n",
      "\n",
      "2019-01-14 18:20:13 (8.09 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]\n",
      "\n"
     ]
    }
   ],
   "source": [
    "%mkdir ../data\n",
    "!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz\n",
    "!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 2: Preparing and Processing the data\n",
    "\n",
    "The data we have downloaded is split into various files, each of which contains a single review. It will be much easier going forward if we combine these individual files into two large files, one for training and one for testing."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import glob\n",
    "\n",
    "def read_imdb_data(data_dir='../data/aclImdb'):\n",
    "    data = {}\n",
    "    labels = {}\n",
    "    \n",
    "    for data_type in ['train', 'test']:\n",
    "        data[data_type] = {}\n",
    "        labels[data_type] = {}\n",
    "        \n",
    "        for sentiment in ['pos', 'neg']:\n",
    "            data[data_type][sentiment] = []\n",
    "            labels[data_type][sentiment] = []\n",
    "            \n",
    "            path = os.path.join(data_dir, data_type, sentiment, '*.txt')\n",
    "            files = glob.glob(path)\n",
    "            \n",
    "            for f in files:\n",
    "                with open(f) as review:\n",
    "                    data[data_type][sentiment].append(review.read())\n",
    "                    # Here we represent a positive review by '1' and a negative review by '0'\n",
    "                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)\n",
    "                    \n",
    "            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \\\n",
    "                    \"{}/{} data size does not match labels size\".format(data_type, sentiment)\n",
    "                \n",
    "    return data, labels"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg\n"
     ]
    }
   ],
   "source": [
    "data, labels = read_imdb_data()\n",
    "print(\"IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg\".format(\n",
    "            len(data['train']['pos']), len(data['train']['neg']),\n",
    "            len(data['test']['pos']), len(data['test']['neg'])))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.utils import shuffle\n",
    "\n",
    "def prepare_imdb_data(data, labels):\n",
    "    \"\"\"Prepare training and test sets from IMDb movie reviews.\"\"\"\n",
    "    \n",
    "    #Combine positive and negative reviews and labels\n",
    "    data_train = data['train']['pos'] + data['train']['neg']\n",
    "    data_test = data['test']['pos'] + data['test']['neg']\n",
    "    labels_train = labels['train']['pos'] + labels['train']['neg']\n",
    "    labels_test = labels['test']['pos'] + labels['test']['neg']\n",
    "    \n",
    "    #Shuffle reviews and corresponding labels within training and test sets\n",
    "    data_train, labels_train = shuffle(data_train, labels_train)\n",
    "    data_test, labels_test = shuffle(data_test, labels_test)\n",
    "    \n",
    "    # Return a unified training data, test data, training labels, test labets\n",
    "    return data_train, data_test, labels_train, labels_test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "IMDb reviews (combined): train = 25000, test = 25000\n"
     ]
    }
   ],
   "source": [
    "train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)\n",
    "print(\"IMDb reviews (combined): train = {}, test = {}\".format(len(train_X), len(test_X)))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Watching Smother was perhaps the longest not-quite-90-minutes of my life. There wasn\\'t a laugh to be had; in fact, I don\\'t remember ever cracking a smile. Diane Keaton was horridly unfunny as a middle-aged chain-smoking dog hoarder, the textbook overbearing mother character, a relentlessly irritating woman who clearly suffers from some kind of personality disorder. She is manipulative, conniving, melodramatic, childish, narcissistic, and worst of all, boring.<br /><br />I suppose I should briefly mention the other characters, but why bother? It was just a long string of movie clichés--the dippy, socially inept distant relative who\\'s just trying to break into \"The Industry\", the gruff and long-suffering but somehow still lovable father, the mild- mannered wife who just can\\'t take it anymore (but eventually moves beyond the discord and resignedly comes home), the herd of unhousebroken dogs who like to chew throw pillows while everyone is away, etc.<br /><br />God, what a snore. I\\'ve never been a Diane Keaton fan and Smother only reminded me why. Overacting is overacting, no matter how many pictures you did in your prime. Her attempts at physical comedy were especially humiliating. What was the director thinking?<br /><br />While I like Dax Shepard and can even sometimes tolerate Liv Tyler, their performances were so lackluster and dull that it was clear that neither actor gave a damn about this movie. That was okay, because neither did I. Keaton\\'s endless self-absorbed prattling was intolerable and at times Shepard\\'s dislike for her seemed genuine. By the end of the movie I wanted to slap her myself.<br /><br />Awful.'"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "train_X[100]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Processing the data\n",
    "\n",
    "Now that we have our training and testing datasets merged and ready to use, we need to start processing the raw data into something that will be useable by our machine learning algorithm. To begin with, we remove any html formatting and any non-alpha numeric characters that may appear in the reviews. We will do this in a very simplistic way using Python's regular expression module. We will discuss the reason for this rather simplistic pre-processing later on."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "import re\n",
    "\n",
    "REPLACE_NO_SPACE = re.compile(\"(\\.)|(\\;)|(\\:)|(\\!)|(\\')|(\\?)|(\\,)|(\\\")|(\\()|(\\))|(\\[)|(\\])\")\n",
    "REPLACE_WITH_SPACE = re.compile(\"(<br\\s*/><br\\s*/>)|(\\-)|(\\/)\")\n",
    "\n",
    "def review_to_words(review):\n",
    "    words = REPLACE_NO_SPACE.sub(\"\", review.lower())\n",
    "    words = REPLACE_WITH_SPACE.sub(\" \", words)\n",
    "    return words"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'watching smother was perhaps the longest not quite 90 minutes of my life there wasnt a laugh to be had in fact i dont remember ever cracking a smile diane keaton was horridly unfunny as a middle aged chain smoking dog hoarder the textbook overbearing mother character a relentlessly irritating woman who clearly suffers from some kind of personality disorder she is manipulative conniving melodramatic childish narcissistic and worst of all boring i suppose i should briefly mention the other characters but why bother it was just a long string of movie clichés  the dippy socially inept distant relative whos just trying to break into the industry the gruff and long suffering but somehow still lovable father the mild  mannered wife who just cant take it anymore but eventually moves beyond the discord and resignedly comes home the herd of unhousebroken dogs who like to chew throw pillows while everyone is away etc god what a snore ive never been a diane keaton fan and smother only reminded me why overacting is overacting no matter how many pictures you did in your prime her attempts at physical comedy were especially humiliating what was the director thinking while i like dax shepard and can even sometimes tolerate liv tyler their performances were so lackluster and dull that it was clear that neither actor gave a damn about this movie that was okay because neither did i keatons endless self absorbed prattling was intolerable and at times shepards dislike for her seemed genuine by the end of the movie i wanted to slap her myself awful'"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "review_to_words(train_X[100])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pickle\n",
    "\n",
    "cache_dir = os.path.join(\"../cache\", \"sentiment_web_app\")  # where to store cache files\n",
    "os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists\n",
    "\n",
    "def preprocess_data(data_train, data_test, labels_train, labels_test,\n",
    "                    cache_dir=cache_dir, cache_file=\"preprocessed_data.pkl\"):\n",
    "    \"\"\"Convert each review to words; read from cache if available.\"\"\"\n",
    "\n",
    "    # If cache_file is not None, try to read from it first\n",
    "    cache_data = None\n",
    "    if cache_file is not None:\n",
    "        try:\n",
    "            with open(os.path.join(cache_dir, cache_file), \"rb\") as f:\n",
    "                cache_data = pickle.load(f)\n",
    "            print(\"Read preprocessed data from cache file:\", cache_file)\n",
    "        except:\n",
    "            pass  # unable to read from cache, but that's okay\n",
    "    \n",
    "    # If cache is missing, then do the heavy lifting\n",
    "    if cache_data is None:\n",
    "        # Preprocess training and test data to obtain words for each review\n",
    "        #words_train = list(map(review_to_words, data_train))\n",
    "        #words_test = list(map(review_to_words, data_test))\n",
    "        words_train = [review_to_words(review) for review in data_train]\n",
    "        words_test = [review_to_words(review) for review in data_test]\n",
    "        \n",
    "        # Write to cache file for future runs\n",
    "        if cache_file is not None:\n",
    "            cache_data = dict(words_train=words_train, words_test=words_test,\n",
    "                              labels_train=labels_train, labels_test=labels_test)\n",
    "            with open(os.path.join(cache_dir, cache_file), \"wb\") as f:\n",
    "                pickle.dump(cache_data, f)\n",
    "            print(\"Wrote preprocessed data to cache file:\", cache_file)\n",
    "    else:\n",
    "        # Unpack data loaded from cache file\n",
    "        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],\n",
    "                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])\n",
    "    \n",
    "    return words_train, words_test, labels_train, labels_test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Wrote preprocessed data to cache file: preprocessed_data.pkl\n"
     ]
    }
   ],
   "source": [
    "# Preprocess data\n",
    "train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Extract Bag-of-Words features\n",
    "\n",
    "For the model we will be implementing, rather than using the reviews directly, we are going to transform each review into a Bag-of-Words feature representation. Keep in mind that 'in the wild' we will only have access to the training set so our transformer can only use the training set to construct a representation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "from sklearn.feature_extraction.text import CountVectorizer\n",
    "from sklearn.externals import joblib\n",
    "# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays\n",
    "\n",
    "def extract_BoW_features(words_train, words_test, vocabulary_size=5000,\n",
    "                         cache_dir=cache_dir, cache_file=\"bow_features.pkl\"):\n",
    "    \"\"\"Extract Bag-of-Words for a given set of documents, already preprocessed into words.\"\"\"\n",
    "    \n",
    "    # If cache_file is not None, try to read from it first\n",
    "    cache_data = None\n",
    "    if cache_file is not None:\n",
    "        try:\n",
    "            with open(os.path.join(cache_dir, cache_file), \"rb\") as f:\n",
    "                cache_data = joblib.load(f)\n",
    "            print(\"Read features from cache file:\", cache_file)\n",
    "        except:\n",
    "            pass  # unable to read from cache, but that's okay\n",
    "    \n",
    "    # If cache is missing, then do the heavy lifting\n",
    "    if cache_data is None:\n",
    "        # Fit a vectorizer to training documents and use it to transform them\n",
    "        # NOTE: Training documents have already been preprocessed and tokenized into words;\n",
    "        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x\n",
    "        vectorizer = CountVectorizer(max_features=vocabulary_size)\n",
    "        features_train = vectorizer.fit_transform(words_train).toarray()\n",
    "\n",
    "        # Apply the same vectorizer to transform the test documents (ignore unknown words)\n",
    "        features_test = vectorizer.transform(words_test).toarray()\n",
    "        \n",
    "        # NOTE: Remember to convert the features using .toarray() for a compact representation\n",
    "        \n",
    "        # Write to cache file for future runs (store vocabulary as well)\n",
    "        if cache_file is not None:\n",
    "            vocabulary = vectorizer.vocabulary_\n",
    "            cache_data = dict(features_train=features_train, features_test=features_test,\n",
    "                             vocabulary=vocabulary)\n",
    "            with open(os.path.join(cache_dir, cache_file), \"wb\") as f:\n",
    "                joblib.dump(cache_data, f)\n",
    "            print(\"Wrote features to cache file:\", cache_file)\n",
    "    else:\n",
    "        # Unpack data loaded from cache file\n",
    "        features_train, features_test, vocabulary = (cache_data['features_train'],\n",
    "                cache_data['features_test'], cache_data['vocabulary'])\n",
    "    \n",
    "    # Return both the extracted features as well as the vocabulary\n",
    "    return features_train, features_test, vocabulary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Wrote features to cache file: bow_features.pkl\n"
     ]
    }
   ],
   "source": [
    "# Extract Bag of Words features for both training and test datasets\n",
    "train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "5000"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(train_X[100])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 3: Upload data to S3\n",
    "\n",
    "Now that we have created the feature representation of our training (and testing) data, it is time to start setting up and using the XGBoost classifier provided by SageMaker.\n",
    "\n",
    "### Writing the datasets\n",
    "\n",
    "The XGBoost classifier that we will be using requires the dataset to be written to a file and stored using Amazon S3. To do this, we will start by splitting the training dataset into two parts, the data we will train the model with and a validation set. Then, we will write those datasets to a file locally and then upload the files to S3. In addition, we will write the test set to a file and upload that file to S3. This is so that we can use SageMakers Batch Transform functionality to test our model once we've fit it."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "\n",
    "# Earlier we shuffled the training dataset so to make things simple we can just assign\n",
    "# the first 10 000 reviews to the validation set and use the remaining reviews for training.\n",
    "val_X = pd.DataFrame(train_X[:10000])\n",
    "train_X = pd.DataFrame(train_X[10000:])\n",
    "\n",
    "val_y = pd.DataFrame(train_y[:10000])\n",
    "train_y = pd.DataFrame(train_y[10000:])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The documentation for the XGBoost algorithm in SageMaker requires that the training and validation datasets should contain no headers or index and that the label should occur first for each sample.\n",
    "\n",
    "For more information about this and other algorithms, the SageMaker developer documentation can be found on __[Amazon's website.](https://docs.aws.amazon.com/sagemaker/latest/dg/)__"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [],
   "source": [
    "# First we make sure that the local directory in which we'd like to store the training and validation csv files exists.\n",
    "data_dir = '../data/sentiment_web_app'\n",
    "if not os.path.exists(data_dir):\n",
    "    os.makedirs(data_dir)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [],
   "source": [
    "pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)\n",
    "\n",
    "pd.concat([val_y, val_X], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)\n",
    "pd.concat([train_y, train_X], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [],
   "source": [
    "# To save a bit of memory we can set text_X, train_X, val_X, train_y and val_y to None.\n",
    "\n",
    "test_X = train_X = val_X = train_y = val_y = None"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Uploading Training / Validation files to S3\n",
    "\n",
    "Amazon's S3 service allows us to store files that can be accessed by both the built-in training models such as the XGBoost model we will be using as well as custom models such as the one we will see a little later.\n",
    "\n",
    "For this and most other tasks we will be doing using SageMaker, there are two methods we could use. The first is to use the low level functionality of SageMaker which requires knowing each of the objects involved in the SageMaker environment. The second is to use the high level functionality in which certain choices have been made on the user's behalf. The low level approach benefits from allowing the user a great deal of flexibility while the high level approach makes development much quicker. For our purposes we will opt to use the high level approach although using the low-level approach is certainly an option.\n",
    "\n",
    "Recall the method `upload_data()` which is a member of the object representing our current SageMaker session. What this method does is upload the data to the default bucket (which is created if it does not exist) into the path described by the key_prefix variable. To see this for yourself, once you have uploaded the data files, go to the S3 console and look to see where the files have been uploaded.\n",
    "\n",
    "For additional resources, see the __[SageMaker API documentation](http://sagemaker.readthedocs.io/en/latest/)__ and in addition the __[SageMaker Developer Guide.](https://docs.aws.amazon.com/sagemaker/latest/dg/)__"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sagemaker\n",
    "\n",
    "session = sagemaker.Session() # Store the current SageMaker session\n",
    "\n",
    "# S3 prefix (which folder will we use)\n",
    "prefix = 'sentiment-web-app'\n",
    "\n",
    "test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)\n",
    "val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)\n",
    "train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 4: Creating the XGBoost model\n",
    "\n",
    "Now that the data has been uploaded it is time to create the XGBoost model. To begin with, we need to do some setup. At this point it is worth discussing what a model is in SageMaker. It is easiest to think of a model of comprising three different objects in the SageMaker ecosystem, which interact with one another.\n",
    "\n",
    "- Model Artifacts\n",
    "- Training Code (Container)\n",
    "- Inference Code (Container)\n",
    "\n",
    "The Model Artifacts are what you might think of as the actual model itself. For example, if you were building a neural network, the model artifacts would be the weights of the various layers. In our case, for an XGBoost model, the artifacts are the actual trees that are created during training.\n",
    "\n",
    "The other two objects, the training code and the inference code are then used to manipulate the training artifacts. More precisely, the training code uses the training data that is provided and creates the model artifacts, while the inference code uses the model artifacts to make predictions on new data.\n",
    "\n",
    "The way that SageMaker runs the training and inference code is by making use of Docker containers. For now, think of a container as being a way of packaging code up so that dependencies aren't an issue."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sagemaker import get_execution_role\n",
    "\n",
    "# Our current execution role is required when creating the model as the training\n",
    "# and inference code will need to access the model artifacts.\n",
    "role = get_execution_role()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [],
   "source": [
    "# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.\n",
    "# As a matter of convenience, the training and inference code both use the same container.\n",
    "from sagemaker.amazon.amazon_estimator import get_image_uri\n",
    "\n",
    "container = get_image_uri(session.boto_region_name, 'xgboost')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "# First we create a SageMaker estimator object for our model.\n",
    "xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use\n",
    "                                    role,                                    # What is our current IAM Role\n",
    "                                    train_instance_count=1,                  # How many compute instances\n",
    "                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances\n",
    "                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),\n",
    "                                    sagemaker_session=session)\n",
    "\n",
    "# And then set the algorithm specific parameters.\n",
    "xgb.set_hyperparameters(max_depth=5,\n",
    "                        eta=0.2,\n",
    "                        gamma=4,\n",
    "                        min_child_weight=6,\n",
    "                        subsample=0.8,\n",
    "                        silent=0,\n",
    "                        objective='binary:logistic',\n",
    "                        early_stopping_rounds=10,\n",
    "                        num_round=500)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Fit the XGBoost model\n",
    "\n",
    "Now that our model has been set up we simply need to attach the training and validation datasets and then ask SageMaker to set up the computation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [],
   "source": [
    "s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')\n",
    "s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "INFO:sagemaker:Creating training-job with name: xgboost-2019-01-14-18-26-14-053\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "2019-01-14 18:26:14 Starting - Starting the training job...\n",
      "2019-01-14 18:26:15 Starting - Launching requested ML instances...\n",
      "2019-01-14 18:27:13 Starting - Preparing the instances for training............\n",
      "2019-01-14 18:29:04 Downloading - Downloading input data\n",
      "2019-01-14 18:29:04 Training - Training image download completed. Training in progress.\n",
      "\u001b[31mArguments: train\u001b[0m\n",
      "\u001b[31m[2019-01-14:18:29:05:INFO] Running standalone xgboost training.\u001b[0m\n",
      "\u001b[31m[2019-01-14:18:29:05:INFO] File size need to be processed in the node: 238.5mb. Available memory size in the node: 8387.41mb\u001b[0m\n",
      "\u001b[31m[2019-01-14:18:29:05:INFO] Determined delimiter of CSV input is ','\u001b[0m\n",
      "\u001b[31m[18:29:05] S3DistributionType set as FullyReplicated\u001b[0m\n",
      "\u001b[31m[18:29:07] 15000x5000 matrix with 75000000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,\u001b[0m\n",
      "\u001b[31m[2019-01-14:18:29:07:INFO] Determined delimiter of CSV input is ','\u001b[0m\n",
      "\u001b[31m[18:29:07] S3DistributionType set as FullyReplicated\u001b[0m\n",
      "\u001b[31m[18:29:08] 10000x5000 matrix with 50000000 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,\u001b[0m\n",
      "\u001b[31m[18:29:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[0]#011train-error:0.3032#011validation-error:0.3107\u001b[0m\n",
      "\u001b[31mMultiple eval metrics have been passed: 'validation-error' will be used for early stopping.\n",
      "\u001b[0m\n",
      "\u001b[31mWill train until validation-error hasn't improved in 10 rounds.\u001b[0m\n",
      "\u001b[31m[18:29:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[1]#011train-error:0.2908#011validation-error:0.2968\u001b[0m\n",
      "\u001b[31m[18:29:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[2]#011train-error:0.283467#011validation-error:0.2947\u001b[0m\n",
      "\u001b[31m[18:29:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[3]#011train-error:0.273133#011validation-error:0.2832\u001b[0m\n",
      "\u001b[31m[18:29:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[4]#011train-error:0.258667#011validation-error:0.2724\u001b[0m\n",
      "\u001b[31m[18:29:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[5]#011train-error:0.247867#011validation-error:0.2653\u001b[0m\n",
      "\u001b[31m[18:29:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[6]#011train-error:0.2486#011validation-error:0.2643\u001b[0m\n",
      "\u001b[31m[18:29:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[7]#011train-error:0.241933#011validation-error:0.2566\u001b[0m\n",
      "\u001b[31m[18:29:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[8]#011train-error:0.237267#011validation-error:0.2515\u001b[0m\n",
      "\u001b[31m[18:29:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[9]#011train-error:0.228133#011validation-error:0.2464\u001b[0m\n",
      "\u001b[31m[18:29:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[10]#011train-error:0.2262#011validation-error:0.2435\u001b[0m\n",
      "\u001b[31m[18:29:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[11]#011train-error:0.219133#011validation-error:0.2385\u001b[0m\n",
      "\u001b[31m[18:29:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 18 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[12]#011train-error:0.215867#011validation-error:0.2333\u001b[0m\n",
      "\u001b[31m[18:29:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[13]#011train-error:0.212667#011validation-error:0.2313\u001b[0m\n",
      "\u001b[31m[18:29:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[14]#011train-error:0.208333#011validation-error:0.2271\u001b[0m\n",
      "\u001b[31m[18:29:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[15]#011train-error:0.2044#011validation-error:0.223\u001b[0m\n",
      "\u001b[31m[18:29:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[16]#011train-error:0.201733#011validation-error:0.2207\u001b[0m\n",
      "\u001b[31m[18:29:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[17]#011train-error:0.198933#011validation-error:0.2188\u001b[0m\n",
      "\u001b[31m[18:29:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[18]#011train-error:0.197#011validation-error:0.2162\u001b[0m\n",
      "\u001b[31m[18:29:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[19]#011train-error:0.194#011validation-error:0.2138\u001b[0m\n",
      "\u001b[31m[18:29:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[20]#011train-error:0.191467#011validation-error:0.2132\u001b[0m\n",
      "\u001b[31m[18:29:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[21]#011train-error:0.188533#011validation-error:0.2112\u001b[0m\n",
      "\u001b[31m[18:29:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[22]#011train-error:0.185467#011validation-error:0.2084\u001b[0m\n",
      "\u001b[31m[18:29:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[23]#011train-error:0.1812#011validation-error:0.2069\u001b[0m\n",
      "\u001b[31m[18:29:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[24]#011train-error:0.177333#011validation-error:0.2047\u001b[0m\n",
      "\u001b[31m[18:29:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[25]#011train-error:0.1752#011validation-error:0.2013\u001b[0m\n",
      "\u001b[31m[18:29:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[26]#011train-error:0.1736#011validation-error:0.2\u001b[0m\n",
      "\u001b[31m[18:29:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[27]#011train-error:0.172#011validation-error:0.1983\u001b[0m\n",
      "\u001b[31m[18:29:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[28]#011train-error:0.169#011validation-error:0.1965\u001b[0m\n",
      "\u001b[31m[18:29:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[29]#011train-error:0.1674#011validation-error:0.1947\u001b[0m\n",
      "\u001b[31m[18:29:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[30]#011train-error:0.167133#011validation-error:0.1931\u001b[0m\n",
      "\u001b[31m[18:29:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[31]#011train-error:0.165333#011validation-error:0.1908\u001b[0m\n",
      "\u001b[31m[18:29:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[32]#011train-error:0.162467#011validation-error:0.1917\u001b[0m\n",
      "\u001b[31m[18:29:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[33]#011train-error:0.1604#011validation-error:0.1912\u001b[0m\n",
      "\u001b[31m[18:29:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[34]#011train-error:0.159867#011validation-error:0.1905\u001b[0m\n",
      "\u001b[31m[18:29:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[35]#011train-error:0.157933#011validation-error:0.1895\u001b[0m\n",
      "\u001b[31m[18:30:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[36]#011train-error:0.156133#011validation-error:0.1888\u001b[0m\n",
      "\u001b[31m[18:30:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[37]#011train-error:0.1544#011validation-error:0.1875\u001b[0m\n",
      "\u001b[31m[18:30:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[38]#011train-error:0.153667#011validation-error:0.1881\u001b[0m\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\u001b[31m[18:30:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[39]#011train-error:0.152067#011validation-error:0.1863\u001b[0m\n",
      "\u001b[31m[18:30:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[40]#011train-error:0.151867#011validation-error:0.1846\u001b[0m\n",
      "\u001b[31m[18:30:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[41]#011train-error:0.1498#011validation-error:0.1836\u001b[0m\n",
      "\u001b[31m[18:30:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 14 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[42]#011train-error:0.148533#011validation-error:0.1823\u001b[0m\n",
      "\u001b[31m[18:30:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[43]#011train-error:0.147333#011validation-error:0.1811\u001b[0m\n",
      "\u001b[31m[18:30:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[44]#011train-error:0.145067#011validation-error:0.18\u001b[0m\n",
      "\u001b[31m[18:30:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[45]#011train-error:0.144133#011validation-error:0.1772\u001b[0m\n",
      "\u001b[31m[18:30:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[46]#011train-error:0.1418#011validation-error:0.1773\u001b[0m\n",
      "\u001b[31m[18:30:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[47]#011train-error:0.140267#011validation-error:0.1774\u001b[0m\n",
      "\u001b[31m[18:30:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[48]#011train-error:0.139267#011validation-error:0.1769\u001b[0m\n",
      "\u001b[31m[18:30:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[49]#011train-error:0.137333#011validation-error:0.1764\u001b[0m\n",
      "\u001b[31m[18:30:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[50]#011train-error:0.136267#011validation-error:0.1736\u001b[0m\n",
      "\u001b[31m[18:30:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[51]#011train-error:0.136#011validation-error:0.1737\u001b[0m\n",
      "\u001b[31m[18:30:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[52]#011train-error:0.134733#011validation-error:0.1726\u001b[0m\n",
      "\u001b[31m[18:30:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[53]#011train-error:0.133467#011validation-error:0.1733\u001b[0m\n",
      "\u001b[31m[18:30:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[54]#011train-error:0.132667#011validation-error:0.1734\u001b[0m\n",
      "\u001b[31m[18:30:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[55]#011train-error:0.131267#011validation-error:0.1731\u001b[0m\n",
      "\u001b[31m[18:30:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 14 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[56]#011train-error:0.1314#011validation-error:0.1721\u001b[0m\n",
      "\u001b[31m[18:30:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 14 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[57]#011train-error:0.130933#011validation-error:0.1721\u001b[0m\n",
      "\u001b[31m[18:30:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[58]#011train-error:0.129533#011validation-error:0.1717\u001b[0m\n",
      "\u001b[31m[18:30:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[59]#011train-error:0.128933#011validation-error:0.1704\u001b[0m\n",
      "\u001b[31m[18:30:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[60]#011train-error:0.1278#011validation-error:0.1703\u001b[0m\n",
      "\u001b[31m[18:30:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[61]#011train-error:0.1264#011validation-error:0.1692\u001b[0m\n",
      "\u001b[31m[18:30:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[62]#011train-error:0.126133#011validation-error:0.1679\u001b[0m\n",
      "\u001b[31m[18:30:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[63]#011train-error:0.124533#011validation-error:0.1678\u001b[0m\n",
      "\u001b[31m[18:30:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[64]#011train-error:0.123533#011validation-error:0.1685\u001b[0m\n",
      "\u001b[31m[18:30:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[65]#011train-error:0.1236#011validation-error:0.1673\u001b[0m\n",
      "\u001b[31m[18:30:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[66]#011train-error:0.124#011validation-error:0.1666\u001b[0m\n",
      "\u001b[31m[18:30:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[67]#011train-error:0.122933#011validation-error:0.1664\u001b[0m\n",
      "\u001b[31m[18:30:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[68]#011train-error:0.122267#011validation-error:0.1659\u001b[0m\n",
      "\u001b[31m[18:30:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 14 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[69]#011train-error:0.121467#011validation-error:0.1655\u001b[0m\n",
      "\u001b[31m[18:30:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[70]#011train-error:0.120667#011validation-error:0.1658\u001b[0m\n",
      "\u001b[31m[18:30:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[71]#011train-error:0.120533#011validation-error:0.1649\u001b[0m\n",
      "\u001b[31m[18:30:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[72]#011train-error:0.119333#011validation-error:0.1648\u001b[0m\n",
      "\u001b[31m[18:30:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[73]#011train-error:0.12#011validation-error:0.1645\u001b[0m\n",
      "\u001b[31m[18:30:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[74]#011train-error:0.119133#011validation-error:0.1632\u001b[0m\n",
      "\u001b[31m[18:30:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[75]#011train-error:0.118267#011validation-error:0.1625\u001b[0m\n",
      "\u001b[31m[18:30:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[76]#011train-error:0.118067#011validation-error:0.1624\u001b[0m\n",
      "\u001b[31m[18:30:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 14 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[77]#011train-error:0.116933#011validation-error:0.1613\u001b[0m\n",
      "\u001b[31m[18:30:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[78]#011train-error:0.116467#011validation-error:0.1607\u001b[0m\n",
      "\u001b[31m[18:30:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[79]#011train-error:0.1148#011validation-error:0.1605\u001b[0m\n",
      "\u001b[31m[18:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[80]#011train-error:0.114667#011validation-error:0.1595\u001b[0m\n",
      "\u001b[31m[18:30:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 16 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[81]#011train-error:0.114733#011validation-error:0.16\u001b[0m\n",
      "\u001b[31m[18:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[82]#011train-error:0.115#011validation-error:0.1596\u001b[0m\n",
      "\u001b[31m[18:31:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[83]#011train-error:0.1142#011validation-error:0.159\u001b[0m\n",
      "\u001b[31m[18:31:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[84]#011train-error:0.1126#011validation-error:0.1589\u001b[0m\n",
      "\u001b[31m[18:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[85]#011train-error:0.112#011validation-error:0.1589\u001b[0m\n",
      "\u001b[31m[18:31:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[86]#011train-error:0.1114#011validation-error:0.1585\u001b[0m\n",
      "\u001b[31m[18:31:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 18 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[87]#011train-error:0.110933#011validation-error:0.1581\u001b[0m\n",
      "\u001b[31m[18:31:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[88]#011train-error:0.110267#011validation-error:0.1578\u001b[0m\n",
      "\u001b[31m[18:31:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[89]#011train-error:0.109667#011validation-error:0.1578\u001b[0m\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\u001b[31m[18:31:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[90]#011train-error:0.1092#011validation-error:0.1568\u001b[0m\n",
      "\u001b[31m[18:31:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[91]#011train-error:0.108533#011validation-error:0.1565\u001b[0m\n",
      "\u001b[31m[18:31:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[92]#011train-error:0.108067#011validation-error:0.1562\u001b[0m\n",
      "\u001b[31m[18:31:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[93]#011train-error:0.107533#011validation-error:0.1555\u001b[0m\n",
      "\u001b[31m[18:31:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[94]#011train-error:0.106267#011validation-error:0.1539\u001b[0m\n",
      "\u001b[31m[18:31:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[95]#011train-error:0.105333#011validation-error:0.1532\u001b[0m\n",
      "\u001b[31m[18:31:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[96]#011train-error:0.104733#011validation-error:0.1536\u001b[0m\n",
      "\u001b[31m[18:31:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[97]#011train-error:0.103867#011validation-error:0.154\u001b[0m\n",
      "\u001b[31m[18:31:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[98]#011train-error:0.103133#011validation-error:0.1538\u001b[0m\n",
      "\u001b[31m[18:31:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[99]#011train-error:0.102267#011validation-error:0.1533\u001b[0m\n",
      "\u001b[31m[18:31:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[100]#011train-error:0.102133#011validation-error:0.1529\u001b[0m\n",
      "\u001b[31m[18:31:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[101]#011train-error:0.100733#011validation-error:0.1529\u001b[0m\n",
      "\u001b[31m[18:31:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[102]#011train-error:0.1008#011validation-error:0.1526\u001b[0m\n",
      "\u001b[31m[18:31:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[103]#011train-error:0.100667#011validation-error:0.1522\u001b[0m\n",
      "\u001b[31m[18:31:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[104]#011train-error:0.0996#011validation-error:0.1514\u001b[0m\n",
      "\u001b[31m[18:31:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[105]#011train-error:0.099533#011validation-error:0.1515\u001b[0m\n",
      "\u001b[31m[18:31:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[106]#011train-error:0.099333#011validation-error:0.1509\u001b[0m\n",
      "\u001b[31m[18:31:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[107]#011train-error:0.098133#011validation-error:0.1507\u001b[0m\n",
      "\u001b[31m[18:31:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[108]#011train-error:0.097133#011validation-error:0.1496\u001b[0m\n",
      "\u001b[31m[18:31:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 14 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[109]#011train-error:0.097467#011validation-error:0.1491\u001b[0m\n",
      "\u001b[31m[18:31:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[110]#011train-error:0.096933#011validation-error:0.149\u001b[0m\n",
      "\u001b[31m[18:31:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[111]#011train-error:0.096333#011validation-error:0.149\u001b[0m\n",
      "\u001b[31m[18:31:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[112]#011train-error:0.095867#011validation-error:0.1492\u001b[0m\n",
      "\u001b[31m[18:31:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[113]#011train-error:0.095133#011validation-error:0.1488\u001b[0m\n",
      "\u001b[31m[18:31:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[114]#011train-error:0.094733#011validation-error:0.1491\u001b[0m\n",
      "\u001b[31m[18:31:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[115]#011train-error:0.095267#011validation-error:0.1482\u001b[0m\n",
      "\u001b[31m[18:31:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[116]#011train-error:0.095267#011validation-error:0.1483\u001b[0m\n",
      "\u001b[31m[18:31:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[117]#011train-error:0.094133#011validation-error:0.148\u001b[0m\n",
      "\u001b[31m[18:31:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[118]#011train-error:0.0932#011validation-error:0.1478\u001b[0m\n",
      "\u001b[31m[18:31:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[119]#011train-error:0.0934#011validation-error:0.1476\u001b[0m\n",
      "\u001b[31m[18:31:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[120]#011train-error:0.092733#011validation-error:0.1477\u001b[0m\n",
      "\u001b[31m[18:31:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[121]#011train-error:0.092533#011validation-error:0.1472\u001b[0m\n",
      "\u001b[31m[18:31:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[122]#011train-error:0.092267#011validation-error:0.1472\u001b[0m\n",
      "\u001b[31m[18:31:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[123]#011train-error:0.091667#011validation-error:0.1468\u001b[0m\n",
      "\u001b[31m[18:31:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[124]#011train-error:0.090267#011validation-error:0.1462\u001b[0m\n",
      "\u001b[31m[18:31:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[125]#011train-error:0.090133#011validation-error:0.1456\u001b[0m\n",
      "\u001b[31m[18:31:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[126]#011train-error:0.090067#011validation-error:0.1456\u001b[0m\n",
      "\u001b[31m[18:31:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[127]#011train-error:0.09#011validation-error:0.1452\u001b[0m\n",
      "\u001b[31m[18:32:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[128]#011train-error:0.088667#011validation-error:0.1457\u001b[0m\n",
      "\u001b[31m[18:32:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[129]#011train-error:0.088733#011validation-error:0.146\u001b[0m\n",
      "\u001b[31m[18:32:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[130]#011train-error:0.088533#011validation-error:0.146\u001b[0m\n",
      "\u001b[31m[18:32:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 14 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[131]#011train-error:0.087533#011validation-error:0.1458\u001b[0m\n",
      "\u001b[31m[18:32:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[132]#011train-error:0.087667#011validation-error:0.1462\u001b[0m\n",
      "\u001b[31m[18:32:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[133]#011train-error:0.087933#011validation-error:0.1449\u001b[0m\n",
      "\u001b[31m[18:32:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[134]#011train-error:0.087667#011validation-error:0.1451\u001b[0m\n",
      "\u001b[31m[18:32:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[135]#011train-error:0.0872#011validation-error:0.1455\u001b[0m\n",
      "\u001b[31m[18:32:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[136]#011train-error:0.087333#011validation-error:0.1452\u001b[0m\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\u001b[31m[18:32:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[137]#011train-error:0.087733#011validation-error:0.1454\u001b[0m\n",
      "\u001b[31m[18:32:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[138]#011train-error:0.087333#011validation-error:0.1454\u001b[0m\n",
      "\u001b[31m[18:32:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 2 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[139]#011train-error:0.086267#011validation-error:0.1458\u001b[0m\n",
      "\u001b[31m[18:32:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 10 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[140]#011train-error:0.086#011validation-error:0.1457\u001b[0m\n",
      "\u001b[31m[18:32:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 12 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[141]#011train-error:0.085#011validation-error:0.145\u001b[0m\n",
      "\u001b[31m[18:32:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[142]#011train-error:0.085#011validation-error:0.1452\u001b[0m\n",
      "\u001b[31m[18:32:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 8 pruned nodes, max_depth=5\u001b[0m\n",
      "\u001b[31m[143]#011train-error:0.085267#011validation-error:0.1453\u001b[0m\n",
      "\u001b[31mStopping. Best iteration:\u001b[0m\n",
      "\u001b[31m[133]#011train-error:0.087933#011validation-error:0.1449\n",
      "\u001b[0m\n",
      "\n",
      "2019-01-14 18:32:30 Uploading - Uploading generated training model\n",
      "2019-01-14 18:32:30 Completed - Training job completed\n",
      "Billable seconds: 218\n"
     ]
    }
   ],
   "source": [
    "xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 5: Testing the model\n",
    "\n",
    "Now that we've fit our XGBoost model, it's time to see how well it performs. To do this we will use SageMakers Batch Transform functionality. Batch Transform is a convenient way to perform inference on a large dataset in a way that is not realtime. That is, we don't necessarily need to use our model's results immediately and instead we can perform inference on a large number of samples. An example of this in industry might be performing an end of month report. This method of inference can also be useful to us as it means that we can perform inference on our entire test set. \n",
    "\n",
    "To perform a Batch Transformation we need to first create a transformer objects from our trained estimator object."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "INFO:sagemaker:Creating model with name: xgboost-2019-01-14-18-26-14-053\n"
     ]
    }
   ],
   "source": [
    "xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Next we actually perform the transform job. When doing so we need to make sure to specify the type of data we are sending so that it is serialized correctly in the background. In our case we are providing our model with csv data so we specify `text/csv`. Also, if the test data that we have provided is too large to process all at once then we need to specify how the data file should be split up. Since each line is a single entry in our data set we tell SageMaker that it can split the input on each line."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "INFO:sagemaker:Creating transform job with name: xgboost-2019-01-14-18-32-58-849\n"
     ]
    }
   ],
   "source": [
    "xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Currently the transform job is running but it is doing so in the background. Since we wish to wait until the transform job is done and we would like a bit of feedback we can run the `wait()` method."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "..............................................!\n"
     ]
    }
   ],
   "source": [
    "xgb_transformer.wait()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now the transform job has executed and the result, the estimated sentiment of each review, has been saved on S3. Since we would rather work on this file locally we can perform a bit of notebook magic to copy the file to the `data_dir`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 27,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Completed 256.0 KiB/369.3 KiB (2.1 MiB/s) with 1 file(s) remaining\r",
      "Completed 369.3 KiB/369.3 KiB (3.0 MiB/s) with 1 file(s) remaining\r",
      "download: s3://sagemaker-eu-west-1-177473201433/xgboost-2019-01-14-18-32-58-849/test.csv.out to ../data/sentiment_web_app/test.csv.out\r\n"
     ]
    }
   ],
   "source": [
    "!aws s3 cp --recursive $xgb_transformer.output_path $data_dir"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The last step is now to read in the output from our model, convert the output to something a little more usable, in this case we want the sentiment to be either `1` (positive) or `0` (negative), and then compare to the ground truth labels."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 28,
   "metadata": {},
   "outputs": [],
   "source": [
    "predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)\n",
    "predictions = [round(num) for num in predictions.squeeze().values]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.85356"
      ]
     },
     "execution_count": 29,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.metrics import accuracy_score\n",
    "accuracy_score(test_y, predictions)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 6: Deploying the model\n",
    "\n",
    "Once we construct and fit our model, SageMaker stores the resulting model artifacts and we can use those to deploy an endpoint (inference code). To see this, look in the SageMaker console and you should see that a model has been created along with a link to the S3 location where the model artifacts have been stored.\n",
    "\n",
    "Deploying an endpoint is a lot like training the model with a few important differences. The first is that a deployed model doesn't change the model artifacts, so as you send it various testing instances the model won't change. Another difference is that since we aren't performing a fixed computation, as we were in the training step or while performing a batch transform, the compute instance that gets started stays running until we tell it to stop. This is important to note as if we forget and leave it running we will be charged the entire time.\n",
    "\n",
    "In other words **If you are no longer using a deployed endpoint, shut it down!**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "INFO:sagemaker:Creating model with name: xgboost-2019-01-14-18-36-52-850\n",
      "INFO:sagemaker:Creating endpoint with name xgboost-2019-01-14-18-26-14-053\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "---------------------------------------------------------------------------!"
     ]
    }
   ],
   "source": [
    "xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Testing the model (again)\n",
    "\n",
    "Now that we have deployed our endpoint, we can send the testing data to it and get back the inference results. We already did this earlier using the batch transform functionality of SageMaker, however, we will test our model again using the newly deployed endpoint so that we can make sure that it works properly and to get a bit of a feel for how the endpoint works.\n",
    "\n",
    "When using the created endpoint it is important to know that we are limited in the amount of information we can send in each call so we need to break the testing data up into chunks and then send each chunk. Also, we need to serialize our data before we send it to the endpoint to ensure that our data is transmitted properly. Fortunately, SageMaker can do the serialization part for us provided we tell it the format of our data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 31,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sagemaker.predictor import csv_serializer\n",
    "\n",
    "# We need to tell the endpoint what format the data we are sending is in so that SageMaker can perform the serialization.\n",
    "xgb_predictor.content_type = 'text/csv'\n",
    "xgb_predictor.serializer = csv_serializer"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 32,
   "metadata": {},
   "outputs": [],
   "source": [
    "# We split the data into chunks and send each chunk seperately, accumulating the results.\n",
    "\n",
    "def predict(data, rows=512):\n",
    "    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))\n",
    "    predictions = ''\n",
    "    for array in split_array:\n",
    "        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])\n",
    "    \n",
    "    return np.fromstring(predictions[1:], sep=',')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 33,
   "metadata": {},
   "outputs": [],
   "source": [
    "test_X = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None).values\n",
    "\n",
    "predictions = predict(test_X)\n",
    "predictions = [round(num) for num in predictions]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Lastly, we check to see what the accuracy of our model is."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 34,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.85356"
      ]
     },
     "execution_count": 34,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.metrics import accuracy_score\n",
    "accuracy_score(test_y, predictions)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "And the results here should agree with the model testing that we did earlier using the batch transform job.\n",
    "\n",
    "### Cleaning up\n",
    "\n",
    "Now that we've determined that deploying our model works as expected, we are going to shut it down. Remember that the longer the endpoint is left running, the greater the cost and since we have a bit more work to do before we are able to use our endpoint with our simple web app, we should shut everything down."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 35,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "INFO:sagemaker:Deleting endpoint with name: xgboost-2019-01-14-18-26-14-053\n"
     ]
    }
   ],
   "source": [
    "xgb_predictor.delete_endpoint()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 7: Putting our model to work\n",
    "\n",
    "As we've mentioned a few times now, our goal is to have our model deployed and then access it using a very simple web app. The intent is for this web app to take some user submitted data (a review), send it off to our endpoint (the model) and then display the result.\n",
    "\n",
    "However, there is a small catch. Currently the only way we can access the endpoint to send it data is using the SageMaker API. We can, if we wish, expose the actual URL that our model's endpoint is receiving data from, however, if we just send it data ourselves we will not get anything in return. This is because the endpoint created by SageMaker requires the entity accessing it have the correct permissions. So, we would need to somehow authenticate our web app with AWS.\n",
    "\n",
    "Having a website that authenticates to AWS seems a bit beyond the scope of this lesson so we will opt for an alternative approach. Namely, we will create a new endpoint which does not require authentication and which acts as a proxy for the SageMaker endpoint.\n",
    "\n",
    "As an additional constraint, we will try to avoid doing any data processing in the web app itself. Remember that when we constructed and tested our model we started with a movie review, then we simplified it by removing any html formatting and punctuation, then we constructed a bag of words embedding and the resulting vector is what we sent to our model. All of this needs to be done to our user input as well.\n",
    "\n",
    "Fortunately we can do all of this data processing in the backend, using Amazon's Lambda service.\n",
    "\n",
    "<img src=\"Web App Diagram.svg\">\n",
    "\n",
    "The diagram above gives an overview of how the various services will work together. On the far right is the model which we trained above and which will be deployed using SageMaker. On the far left is our web app that collects a user's movie review, sends it off and expects a positive or negative sentiment in return.\n",
    "\n",
    "In the middle is where some of the magic happens. We will construct a Lambda function, which you can think of as a straightforward Python function that can be executed whenever a specified event occurs. This Python function will do the data processing we need to perform on a user submitted review. In addition, we will give this function permission to send and recieve data from a SageMaker endpoint.\n",
    "\n",
    "Lastly, the method we will use to execute the Lambda function is a new endpoint that we will create using API Gateway. This endpoint will be a url that listens for data to be sent to it. Once it gets some data it will pass that data on to the Lambda function and then return whatever the Lambda function returns. Essentially it will act as an interface that lets our web app communicate with the Lambda function.\n",
    "\n",
    "### Processing a single review\n",
    "\n",
    "For now, suppose we are given a movie review by our user in the form of a string, like so:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 36,
   "metadata": {},
   "outputs": [],
   "source": [
    "test_review = \"Nothing but a disgusting materialistic pageant of glistening abed remote control greed zombies, totally devoid of any heart or heat. A romantic comedy that has zero romantic chemestry and zero laughs!\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "How do we go from this string to the bag of words feature vector that is expected by our model?\n",
    "\n",
    "If we recall at the beginning of this notebook, the first step is to remove any unnecessary characters using the `review_to_words` method. Remember that we intentionally did this in a very simplistic way. This is because we are going to have to copy this method to our (eventual) Lambda function (we will go into more detail later) and this means it needs to be rather simplistic."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 37,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "nothing but a disgusting materialistic pageant of glistening abed remote control greed zombies totally devoid of any heart or heat a romantic comedy that has zero romantic chemestry and zero laughs\n"
     ]
    }
   ],
   "source": [
    "test_words = review_to_words(test_review)\n",
    "print(test_words)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Next, we need to construct a bag of words embedding of the `test_words` string. To do this, remember that a bag of words embedding uses a `vocabulary` consisting of the most frequently appearing words in a set of documents. Then, for each word in the vocabulary we record the number of times that word appears in `test_words`. We constructed the `vocabulary` earlier using the training set for our problem so encoding `test_words` is relatively straightforward."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 38,
   "metadata": {},
   "outputs": [],
   "source": [
    "def bow_encoding(words, vocabulary):\n",
    "    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero.\n",
    "    for word in words.split():  # For each word in the string\n",
    "        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.\n",
    "            bow[vocabulary[word]] += 1\n",
    "    return bow"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 39,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]\n"
     ]
    }
   ],
   "source": [
    "test_bow = bow_encoding(test_words, vocabulary)\n",
    "print(test_bow)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 40,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "5000"
      ]
     },
     "execution_count": 40,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(test_bow)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "So now we know how to construct a bag of words encoding of a user provided review, how to we send it to our endpoint? First, we need to start the endpoint back up."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 41,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "INFO:sagemaker:Creating model with name: xgboost-2019-01-14-18-45-26-269\n",
      "INFO:sagemaker:Creating endpoint with name xgboost-2019-01-14-18-26-14-053\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "--------------------------------------------------------------------------!"
     ]
    }
   ],
   "source": [
    "xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "At this point we could just do the same thing that we did earlier when we tested our deployed model and send `test_bow` to our endpoint using the `xgb_predictor` object. However, when we eventually construct our Lambda function we won't have access to this object, so how do we call a SageMaker endpoint?\n",
    "\n",
    "It turns out that Python functions that are used in Lambda have access to another Amazon library called `boto3`. This library provides an API for working with Amazon services, including SageMaker. To start with, we need to get a handle to the SageMaker runtime."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 42,
   "metadata": {},
   "outputs": [],
   "source": [
    "import boto3\n",
    "\n",
    "runtime = boto3.Session().client('sagemaker-runtime')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "And now that we have access to the SageMaker runtime, we can ask it to make use of (invoke) an endpoint that has already been created. However, we need to provide SageMaker with the name of the deployed endpoint. To find this out we can print it out using the `xgb_predictor` object."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 43,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'xgboost-2019-01-14-18-26-14-053'"
      ]
     },
     "execution_count": 43,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "xgb_predictor.endpoint"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Using the SageMaker runtime and the name of our endpoint, we can invoke the endpoint and send it the `test_bow` data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 44,
   "metadata": {},
   "outputs": [
    {
     "ename": "ParamValidationError",
     "evalue": "Parameter validation failed:\nInvalid type for parameter Body, value: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0], type: <class 'list'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object",
     "output_type": "error",
     "traceback": [
      "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[0;31mParamValidationError\u001b[0m                      Traceback (most recent call last)",
      "\u001b[0;32m<ipython-input-44-c9ab54ae3a48>\u001b[0m in \u001b[0;36m<module>\u001b[0;34m()\u001b[0m\n\u001b[1;32m      1\u001b[0m response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created\n\u001b[1;32m      2\u001b[0m                                        \u001b[0mContentType\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;34m'text/csv'\u001b[0m\u001b[0;34m,\u001b[0m                     \u001b[0;31m# The data format that is expected\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m----> 3\u001b[0;31m                                        Body = test_bow)\n\u001b[0m",
      "\u001b[0;32m~/anaconda3/envs/python3/lib/python3.6/site-packages/botocore/client.py\u001b[0m in \u001b[0;36m_api_call\u001b[0;34m(self, *args, **kwargs)\u001b[0m\n\u001b[1;32m    318\u001b[0m                     \"%s() only accepts keyword arguments.\" % py_operation_name)\n\u001b[1;32m    319\u001b[0m             \u001b[0;31m# The \"self\" in this scope is referring to the BaseClient.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 320\u001b[0;31m             \u001b[0;32mreturn\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_make_api_call\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0moperation_name\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    321\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    322\u001b[0m         \u001b[0m_api_call\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m__name__\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mstr\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpy_operation_name\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/anaconda3/envs/python3/lib/python3.6/site-packages/botocore/client.py\u001b[0m in \u001b[0;36m_make_api_call\u001b[0;34m(self, operation_name, api_params)\u001b[0m\n\u001b[1;32m    595\u001b[0m         }\n\u001b[1;32m    596\u001b[0m         request_dict = self._convert_to_request_dict(\n\u001b[0;32m--> 597\u001b[0;31m             api_params, operation_model, context=request_context)\n\u001b[0m\u001b[1;32m    598\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    599\u001b[0m         \u001b[0mservice_id\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_service_model\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mservice_id\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mhyphenize\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/anaconda3/envs/python3/lib/python3.6/site-packages/botocore/client.py\u001b[0m in \u001b[0;36m_convert_to_request_dict\u001b[0;34m(self, api_params, operation_model, context)\u001b[0m\n\u001b[1;32m    643\u001b[0m             api_params, operation_model, context)\n\u001b[1;32m    644\u001b[0m         request_dict = self._serializer.serialize_to_request(\n\u001b[0;32m--> 645\u001b[0;31m             api_params, operation_model)\n\u001b[0m\u001b[1;32m    646\u001b[0m         \u001b[0;32mif\u001b[0m \u001b[0;32mnot\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_client_config\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0minject_host_prefix\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    647\u001b[0m             \u001b[0mrequest_dict\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mpop\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m'host_prefix'\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/anaconda3/envs/python3/lib/python3.6/site-packages/botocore/validate.py\u001b[0m in \u001b[0;36mserialize_to_request\u001b[0;34m(self, parameters, operation_model)\u001b[0m\n\u001b[1;32m    295\u001b[0m                                                     operation_model.input_shape)\n\u001b[1;32m    296\u001b[0m             \u001b[0;32mif\u001b[0m \u001b[0mreport\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mhas_errors\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 297\u001b[0;31m                 \u001b[0;32mraise\u001b[0m \u001b[0mParamValidationError\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mreport\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mreport\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mgenerate_report\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    298\u001b[0m         return self._serializer.serialize_to_request(parameters,\n\u001b[1;32m    299\u001b[0m                                                      operation_model)\n",
      "\u001b[0;31mParamValidationError\u001b[0m: Parameter validation failed:\nInvalid type for parameter Body, value: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0], type: <class 'list'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object"
     ]
    }
   ],
   "source": [
    "response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created\n",
    "                                       ContentType = 'text/csv',                     # The data format that is expected\n",
    "                                       Body = test_bow)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "So why did we get an error?\n",
    "\n",
    "Because we tried to send the endpoint a list of integers but it expected us to send data of type `text/csv`. So, we need to convert it."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 45,
   "metadata": {},
   "outputs": [],
   "source": [
    "response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created\n",
    "                                       ContentType = 'text/csv',                     # The data format that is expected\n",
    "                                       Body = ','.join([str(val) for val in test_bow]).encode('utf-8'))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 46,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "{'ResponseMetadata': {'RequestId': 'e03cc1d2-9bce-4e33-b78f-9699941ad903', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e03cc1d2-9bce-4e33-b78f-9699941ad903', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Mon, 14 Jan 2019 18:53:09 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '14'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f9aa728b5f8>}\n"
     ]
    }
   ],
   "source": [
    "print(response)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "As we can see, the response from our model is a somewhat complicated looking dict that contains a bunch of information. The bit that we are most interested in is `'Body'` object which is a streaming object that we need to `read` in order to make use of."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 47,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.426520615816\n"
     ]
    }
   ],
   "source": [
    "response = response['Body'].read().decode('utf-8')\n",
    "print(response)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now that we know how to process the incoming user data we can start setting up the infrastructure to make our simple web app work. To do this we will make use of two different services. Amazon's Lambda and API Gateway services.\n",
    "\n",
    "Lambda is a service which allows someone to write some relatively simple code and have it executed whenever a chosen trigger occurs. For example, you may want to update a database whenever new data is uploaded to a folder stored on S3.\n",
    "\n",
    "API Gateway is a service that allows you to create HTTP endpoints (url addresses) which are connected to other AWS services. One of the benefits to this is that you get to decide what credentials, if any, are required to access these endpoints.\n",
    "\n",
    "In our case we are going to set up an HTTP endpoint through API Gateway which is open to the public. Then, whenever anyone sends data to our public endpoint we will trigger a Lambda function which will send the input (in our case a review) to our model's endpoint and then return the result."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Setting up a Lambda function\n",
    "\n",
    "The first thing we are going to do is set up a Lambda function. This Lambda function will be executed whenever our public API has data sent to it. When it is executed it will receive the data, perform any sort of processing that is required, send the data (the review) to the SageMaker endpoint we've created and then return the result.\n",
    "\n",
    "#### Part A: Create an IAM Role for the Lambda function\n",
    "\n",
    "Since we want the Lambda function to call a SageMaker endpoint, we need to make sure that it has permission to do so. To do this, we will construct a role that we can later give the Lambda function.\n",
    "\n",
    "Using the AWS Console, navigate to the **IAM** page and click on **Roles**. Then, click on **Create role**. Make sure that the **AWS service** is the type of trusted entity selected and choose **Lambda** as the service that will use this role, then click **Next: Permissions**.\n",
    "\n",
    "In the search box type `sagemaker` and select the check box next to the **AmazonSageMakerFullAccess** policy. Then, click on **Next: Review**.\n",
    "\n",
    "Lastly, give this role a name. Make sure you use a name that you will remember later on, for example `LambdaSageMakerRole`. Then, click on **Create role**."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Part B: Create a Lambda function\n",
    "\n",
    "Now it is time to actually create the Lambda function. Remember from earlier that in order to process the user provided input and send it to our endpoint we need to gather two pieces of information:\n",
    "\n",
    " - The name of the endpoint, and\n",
    " - the vocabulary object.\n",
    "\n",
    "We will copy these pieces of information to our Lambda function after we create it.\n",
    "\n",
    "To start, using the AWS Console, navigate to the AWS Lambda page and click on **Create a function**. When you get to the next page, make sure that **Author from scratch** is selected. Now, name your Lambda function, using a name that you will remember later on, for example `sentiment_analysis_xgboost_func`. Make sure that the **Python 3.6** runtime is selected and then choose the role that you created in the previous part. Then, click on **Create Function**.\n",
    "\n",
    "On the next page you will see some information about the Lambda function you've just created. If you scroll down you should see an editor in which you can write the code that will be executed when your Lambda function is triggered. Collecting the code we wrote above to process a single review and adding it to the provided example `lambda_handler` we arrive at the following.\n",
    "\n",
    "```python\n",
    "# We need to use the low-level library to interact with SageMaker since the SageMaker API\n",
    "# is not available natively through Lambda.\n",
    "import boto3\n",
    "\n",
    "# And we need the regular expression library to do some of the data processing\n",
    "import re\n",
    "\n",
    "REPLACE_NO_SPACE = re.compile(\"(\\.)|(\\;)|(\\:)|(\\!)|(\\')|(\\?)|(\\,)|(\\\")|(\\()|(\\))|(\\[)|(\\])\")\n",
    "REPLACE_WITH_SPACE = re.compile(\"(<br\\s*/><br\\s*/>)|(\\-)|(\\/)\")\n",
    "\n",
    "def review_to_words(review):\n",
    "    words = REPLACE_NO_SPACE.sub(\"\", review.lower())\n",
    "    words = REPLACE_WITH_SPACE.sub(\" \", words)\n",
    "    return words\n",
    "    \n",
    "def bow_encoding(words, vocabulary):\n",
    "    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero.\n",
    "    for word in words.split():  # For each word in the string\n",
    "        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.\n",
    "            bow[vocabulary[word]] += 1\n",
    "    return bow\n",
    "\n",
    "\n",
    "def lambda_handler(event, context):\n",
    "    \n",
    "    vocab = \"*** ACTUAL VOCABULARY GOES HERE ***\"\n",
    "    \n",
    "    words = review_to_words(event['body'])\n",
    "    bow = bow_encoding(words, vocab)\n",
    "\n",
    "    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.\n",
    "    runtime = boto3.Session().client('sagemaker-runtime')\n",
    "\n",
    "    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given\n",
    "    response = runtime.invoke_endpoint(EndpointName = '***ENDPOINT NAME HERE***',# The name of the endpoint we created\n",
    "                                       ContentType = 'text/csv',                 # The data format that is expected\n",
    "                                       Body = ','.join([str(val) for val in bow]).encode('utf-8')) # The actual review\n",
    "\n",
    "    # The response is an HTTP response whose body contains the result of our inference\n",
    "    result = response['Body'].read().decode('utf-8')\n",
    "    \n",
    "    # Round the result so that our web app only gets '1' or '0' as a response.\n",
    "    result = round(float(result))\n",
    "\n",
    "    return {\n",
    "        'statusCode' : 200,\n",
    "        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },\n",
    "        'body' : str(result)\n",
    "    }\n",
    "```"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Once you have copy and pasted the code above into the Lambda code editor, replace the `**ENDPOINT NAME HERE**` portion with the name of the endpoint that we deployed earlier. You can determine the name of the endpoint using the code cell below."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 48,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'xgboost-2019-01-14-18-26-14-053'"
      ]
     },
     "execution_count": 48,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "xgb_predictor.endpoint"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "In addition, you will need to copy the vocabulary dict to the appropriate place in the code at the beginning of the `lambda_handler` method. The cell below prints out the vocabulary dict in a way that is easy to copy and paste."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 49,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "{'this': 4457, 'really': 3556, 'doesnt': 1309, 'match': 2762, 'up': 4683, 'to': 4507, 'castle': 712, 'of': 3083, 'isnt': 2362, 'as': 311, 'funny': 1852, 'or': 3124, 'wacky': 4769, 'the': 4426, 'scenery': 3810, 'and': 226, 'music': 2946, 'are': 285, 'uninspired': 4657, 'plot': 3309, 'just': 2437, 'interesting': 2325, 'only': 3110, 'good': 1937, 'thing': 4450, 'about': 67, 'un': 4630, 'is': 2360, 'nudity': 3059, 'in': 2257, 'version': 4720, 'provided': 3449, 'by': 642, 'it': 2366, 'helped': 2080, 'some': 4061, 'tedious': 4387, 'scenes': 3811, 'had': 2003, 'villain': 4744, 'set': 3898, 'movie': 2926, 'for': 1784, 'imaginative': 2237, 'pieces': 3273, 'locations': 2636, 'not': 3040, 'very': 4723, 'vivid': 4760, 'engaging': 1481, 'dont': 1324, 'even': 1533, 'provide': 3448, 'decent': 1147, 'entertainment': 1500, 'time': 4495, 'its': 2370, 'like': 2599, 'they': 4442, 'were': 4848, 'appearing': 277, 'dvd': 1391, 'full': 1846, 'frame': 1815, 'with': 4911, 'sound': 4081, 'has': 2043, 'amount': 219, 'extras': 1608, 'including': 2264, 'quite': 3495, 'few': 1700, 'trailers': 4555, 'but': 636, 'one': 3107, 'curious': 1083, 'there': 4438, 'no': 3020, 'chapter': 757, 'on': 3105, 'disc': 1278, 'displayed': 1292, 'player': 3298, 'once': 3106, 'though': 4461, 'you': 4986, 'can': 665, 'still': 4188, 'skip': 4016, 'next': 3007, 'scene': 3809, 'number': 3060, 'using': 4698, 'remote': 3622, 'purple': 3473, 'rain': 3506, 'so': 4046, 'cool': 989, 'dad': 1095, 'we': 4823, 'from': 1840, 'hudson': 2184, 'ride': 3692, 'take': 4349, 'lets': 2580, 'go': 1924, 'back': 378, '1984': 23, 'was': 4805, 'released': 3603, 'prince': 3396, 'into': 2332, 'would': 4957, 'think': 4452, 'will': 4886, 'be': 418, 'his': 2118, 'new': 3003, '20th': 38, 'century': 736, 'fox': 1814, 'called': 652, 'starring': 4154, 'voiced': 4762, 'dan': 1102, 'green': 1971, 'who': 4871, 'played': 3297, 'pokemon': 3320, 'leader': 2546, 'tracks': 4545, 'private': 3404, 'dancer': 1104, 'billy': 482, 'suddenly': 4273, 'headed': 2058, 'album': 175, 'held': 2074, 'more': 2904, 'weve': 4856, 'pulled': 3462, 'movies': 2927, 'our': 3139, 'fans': 1650, 'watching': 4815, 'anymore': 258, 'kid': 2464, 'out': 3141, 'look': 2646, 'such': 4268, 'danger': 1109, 'mind': 2849, 'ralph': 3510, 'video': 4733, 'tom': 4514, 'cruise': 1071, 'jumps': 2432, 'brad': 560, 'pitt': 3280, 'thanks': 4423, 'bette': 471, 'heroic': 2090, 'writing': 4971, 'shall': 3916, 'watch': 4811, 'bridge': 578, 'enjoy': 1484, 'kids': 2467, 'written': 4972, 'famous': 1647, 'theatre': 4429, 'film': 1716, 'writer': 4968, 'probably': 3407, 'best': 469, 'ever': 1538, 'made': 2697, 'comedy': 890, 'category': 718, 'yet': 4984, 'appearance': 274, 'play': 3296, 'feature': 1678, 'takes': 4351, 'nothing': 3047, 'value': 4709, 'masterpiece': 2759, 'should': 3951, 'miss': 2866, 'see': 3855, 'subtitles': 4259, 'dubbed': 1379, 'an': 223, 'aged': 149, 'bus': 631, 'capital': 673, 'war': 4792, 'decides': 1150, 'two': 4620, 'sing': 3992, 'their': 4431, 'miserable': 2863, 'life': 2592, 'also': 199, 'tragedy': 4551, 'come': 886, 'singing': 3995, 'both': 547, 'extraordinary': 1607, 'road': 3709, 'real': 3547, 'brothers': 599, 'amateur': 206, 'actors': 104, 'together': 4511, 'whos': 4876, 'give': 1912, 'performance': 3228, 'characters': 760, 'gives': 1914, 'brilliant': 583, 'image': 2233, 'society': 4050, 'facing': 1620, 'insight': 2298, 'nations': 2971, 'person': 3238, 'everyone': 1542, 'aware': 368, 'that': 4424, 'begin': 442, 'try': 4600, 'live': 2625, 'own': 3158, 'lives': 2628, 'hoping': 2160, 'might': 2835, 'simple': 3983, 'language': 2516, 'director': 1264, 'picture': 3269, 'torn': 4528, 'previous': 3387, 'world': 4946, 'highlights': 2104, 'personal': 3240, 'success': 4265, 'singer': 3993, 'great': 1964, 'veteran': 4724, 'way': 4820, 'visit': 4755, 'recently': 3567, 'son': 4070, 'man': 2721, 'seems': 3864, 'german': 1896, 'spy': 4135, 'narration': 2965, 'memorable': 2802, 'soundtrack': 4085, 'songs': 4072, 'atmosphere': 333, 'well': 4843, 'photographed': 3258, 'edited': 1420, 'debut': 1143, 'introduction': 2339, 'series': 3890, 'directors': 1266, 'bitter': 491, 'sweet': 4336, 'comedies': 889, 'cinematography': 820, '1980s': 21, 'sam': 3777, 'my': 2951, 'other': 3135, 'favourite': 1673, 'story': 4207, 'seen': 3865, 'before': 440, 'matter': 2769, 'if': 2222, 'figure': 1711, 'make': 2713, 'proper': 3436, 'clear': 839, 'havent': 2053, 'spent': 4108, 'work': 4940, 'alongside': 196, 'far': 1653, 'too': 4522, 'much': 2934, 'angles': 237, 'do': 1302, 'message': 2819, 'funniest': 1851, 'however': 2183, 'at': 331, 'website': 4834, 'read': 3543, 'purpose': 3474, 'chosen': 805, 'bad': 382, 'camera': 659, 'because': 432, 'remind': 3618, 'us': 4690, 'hunting': 2201, 'have': 2052, 'never': 3001, 'heard': 2062, 'poor': 3328, 'stars': 4155, 'ok': 3098, 'pity': 3281, 'west': 4852, 'review': 3676, 'important': 2247, 'than': 4420, 'planning': 3291, 'does': 1308, 'mean': 2779, 'right': 3696, 'away': 369, 'all': 187, 'word': 4937, 'advice': 133, 'stop': 4201, 'what': 4858, 'want': 4788, 'use': 4692, 'your': 4991, 'start': 4156, 'filming': 1718, 'until': 4680, 'everything': 1544, 'down': 1335, 'certainly': 738, 'ability': 65, 'desire': 1205, 'abuse': 74, 'talent': 4354, 'horrible': 2163, 'most': 2909, 'sci': 3815, 'fi': 1701, 'channel': 753, 'am': 204, 'used': 4693, 'seeing': 3857, 'gray': 1963, 'cgi': 740, 'amateurish': 207, 'special': 4097, 'effects': 1431, 'close': 853, 'ups': 4686, 'fake': 1635, 'blood': 512, 'obvious': 3071, 'thought': 4462, 'possible': 3350, 'could': 1007, 'hit': 2122, 'then': 4436, 'saw': 3798, 'imagine': 2238, 'hand': 2015, 'inside': 2297, 'those': 4460, 'main': 2707, 'alien': 183, 'vampires': 4712, 'clearly': 840, 'talking': 4360, 'comes': 891, 'guts': 1999, 'victims': 4730, 'vampire': 4711, 'hunters': 2200, 'team': 4378, 'talk': 4358, 'these': 4441, 'rubber': 3747, 'how': 2181, 'did': 1241, 'learn': 2550, 'shouldnt': 3952, 'why': 4878, 'little': 2624, 'town': 4541, 'suspend': 4329, 'gag': 1861, 'get': 1899, 'past': 3207, 'moving': 2928, 'aliens': 184, 'heads': 2059, 'deal': 1135, 'wear': 4830, 'faces': 1618, 'plain': 3286, 'ugly': 4625, 'ones': 3108, 'seem': 3861, 'lot': 2661, 'facial': 1619, 'annoying': 249, 'valley': 4707, 'girls': 1911, 'human': 2188, 'sneak': 4043, 'space': 4090, 'defense': 1159, 'attack': 337, 'finally': 1724, 'above': 68, 'worth': 4953, 'laugh': 2529, 'third': 4455, 'rate': 3526, 'cast': 710, 'network': 3000, 'tv': 4610, 'stereotypical': 4180, 'angry': 238, 'heroes': 2089, 'which': 4867, 'constantly': 958, 'fighting': 1709, 'among': 217, 'themselves': 4435, 'almost': 193, 'every': 1539, 'lead': 2545, 'crude': 1069, 'totally': 4532, 'found': 1811, 'myself': 2953, 'win': 4892, 'disappointing': 1274, 'me': 2778, 'came': 656, 'via': 4726, 'classic': 835, 'noir': 3023, 'gift': 1906, 'someone': 4064, 'knows': 2493, 'typically': 4624, 'films': 1721, 'done': 1322, 'style': 4249, 'genre': 1888, 'days': 1130, 'market': 2740, 'black': 493, 'white': 4870, 'always': 203, 'fit': 1740, 'male': 2719, 'protagonist': 3441, 'ed': 1415, 'lee': 2559, 'action': 100, 'place': 3282, 'elements': 1443, 'paranoia': 3186, 'any': 256, 'reason': 3557, 'jane': 2380, 'unconvincing': 4638, 'spoil': 4121, 'potential': 3355, 'viewers': 4739, 'interested': 2324, 'hearing': 2063, 'say': 3799, 'sex': 3907, 'revealing': 3672, 'sexy': 3911, 'plenty': 3307, 'ignore': 2224, 'realize': 3552, 'erotic': 1515, 'red': 3576, 'shoes': 3941, 'junk': 2436, 'hot': 2174, 'exciting': 1563, 'properly': 3437, 'explicit': 1591, 'youll': 4988, 'torture': 4529, 'fest': 1696, 'sit': 4003, 'through': 4477, 'first': 1736, 'mistake': 2871, 'nuclear': 3057, 'power': 3359, 'self': 3869, 'barely': 399, 'needs': 2990, 'convince': 983, 'audience': 352, 'when': 4862, 'putting': 3482, 'argument': 291, 'deliver': 1169, 'facts': 1624, 'thats': 4425, 'missing': 2869, 'crying': 1074, 'going': 1931, 'children': 792, 'while': 4868, 'kept': 2456, 'thinking': 4453, 'hear': 2061, 'cant': 671, 'possibly': 3351, 'true': 4596, 'em': 1452, 'regular': 3592, 'folks': 1771, 'across': 96, 'people': 3221, 'understanding': 4645, 'fair': 1630, 'fault': 1668, 'makers': 2715, 'equally': 1510, 'fairly': 1631, 'condition': 937, 'rock': 3720, 'performers': 3232, 'now': 3054, 'highly': 2105, 'respected': 3653, 'whose': 4877, 'ive': 2372, 'enjoyed': 1486, 'im': 2232, 'sure': 4308, 'them': 4432, 'believe': 454, 'cause': 723, 'old': 3100, 'desperately': 1209, 'searching': 3842, 'something': 4066, 'worked': 4941, 'over': 3148, '60s': 54, 'passed': 3202, 'particularly': 3195, 'embarrassing': 1454, 'james': 2378, 'taylor': 4372, 'trying': 4601, 'hard': 2033, 'he': 2056, 'looks': 2649, 'things': 4451, 'hes': 2093, 'being': 448, 'told': 4513, 'blame': 499, 'him': 2110, 'desperate': 1208, 'feel': 1684, 'noticed': 3049, 'included': 2262, 'performs': 3234, 'stand': 4143, 'fight': 1707, 'fact': 1621, 'performances': 3229, 'pretty': 3384, 'bland': 500, 'theyre': 4445, 'nobody': 3022, 'event': 1535, 'driving': 1368, 'force': 1786, 'behind': 447, 'whole': 4873, 'john': 2409, 'hall': 2009, 'band': 391, 'responsible': 3656, 'pop': 3330, '70s': 56, 'remember': 3616, 'dare': 1113, 'dance': 1103, 'here': 2086, 'fails': 1628, 'passion': 3205, 'supposedly': 4307, 'death': 1141, 'everybody': 1540, 'numbers': 3061, 'playing': 3300, 'except': 1556, 'maybe': 2776, 'scott': 3824, 'lost': 2660, 'topic': 4526, 'least': 2554, 'delivered': 1170, 'high': 2100, 'impressed': 2251, 'controversial': 978, 'bruce': 602, 'been': 438, 'big': 478, 'fan': 1648, 'boss': 546, 'absolutely': 72, 'question': 3487, 'part': 3193, 'thinks': 4454, 'another': 250, 'wake': 4774, 'compared': 912, 'general': 1880, 'air': 164, 'wonder': 4926, 'concert': 935, 'shooting': 3943, 'expect': 1575, 'movement': 2923, 'shots': 3950, 'broken': 594, 'split': 4120, 'second': 3846, 'singers': 3994, 'may': 2775, 'documentaries': 1306, 'footage': 1782, 'last': 2521, 'skill': 4013, 'theres': 4440, 'stellar': 4174, 'sad': 3767, 'counter': 1011, 'culture': 1079, 'gave': 1875, 'many': 2732, 'twists': 4619, 'root': 3737, 'total': 4531, 'gets': 1900, 'killed': 2469, 'nearly': 2981, 'tired': 4501, 'cross': 1066, 'changing': 752, 'views': 4742, 'cannot': 670, 'convoluted': 987, 'insane': 2296, 'paid': 3169, 'demand': 1174, 'money': 2888, 'wish': 4904, 'reviews': 3679, 'honest': 2146, 'worst': 4952, 'original': 3129, 'supposed': 4306, 'makes': 2716, 'fun': 1848, 'line': 2610, 'dinosaur': 1255, 'meat': 2787, 'quote': 3496, 'sounds': 4084, 'pot': 3354, 'dream': 1354, 'writers': 4969, 'after': 143, 'long': 2644, 'night': 3013, 'getting': 1901, 'revolution': 3680, 'ideas': 2215, 'got': 1944, 'planet': 3289, 'dinosaurs': 1256, 'houses': 2180, 'protect': 3443, 'science': 3816, 'idea': 2213, 'fits': 1741, 'complete': 921, 'guns': 1998, 'times': 4497, 'travel': 4567, 'terrible': 4407, 'acting': 99, 'buy': 640, 'al': 170, 'pacino': 3164, 're': 3535, 'sea': 3838, 'love': 2669, 'york': 4985, 'mayor': 2777, 'city': 825, 'thriller': 4472, 'shot': 3949, 'lower': 2677, 'manhattan': 2726, 'structure': 4233, 'ground': 1979, 'zero': 4996, 'government': 1948, 'nyc': 3065, 'settings': 3901, 'cusack': 1088, 'follows': 1775, 'trail': 4553, 'cover': 1022, 'serves': 3895, 'fonda': 1777, 'danny': 1112, 'martin': 2748, 'tony': 4521, 'david': 1124, 'add': 115, 'brilliance': 582, 'gripping': 1976, 'tale': 4353, 'beyond': 476, 'cliché': 844, 'ridden': 3691, 'surprises': 4314, 'list': 2617, 'agree': 155, 'unfunny': 4655, 'whats': 4860, 'worse': 4951, '100': 1, 'million': 2847, 'states': 4163, 'show': 3953, 'pull': 3461, 'bucks': 606, 'felt': 1692, 'sorry': 4076, 'involved': 2346, 'tried': 4584, 'given': 1913, 'rest': 3657, 'seemed': 3862, 'walking': 4779, 'whatever': 4859, 'fall': 1637, 'excellent': 1555, 'saved': 3795, 'actually': 109, 'didnt': 1242, 'camp': 662, 'laughed': 2532, 'parts': 3199, 'touching': 4536, 'overall': 3150, 'know': 2489, 'based': 404, 'book': 535, 'hope': 2155, 'better': 472, 'frankly': 1823, 'boring': 543, 'said': 3774, 'went': 4847, 'bunch': 623, 'excited': 1561, 'plus': 3311, 'drink': 1361, 'nonetheless': 3029, 'stay': 4166, 'awake': 365, 'fear': 1676, 'embarrassed': 1453, 'pointless': 3318, 'name': 2960, 'listed': 2618, 'wrong': 4973, 'opening': 3114, 'credits': 1050, 'credit': 1049, 'reads': 3545, 'cuba': 1075, 'jr': 2423, 'nice': 3008, 'car': 680, 'chase': 769, 'shoot': 3942, 'blow': 514, 'relationship': 3597, 'exists': 1573, 'completely': 922, 'unbelievable': 4634, 'motivations': 2915, 'especially': 1520, 'end': 1468, 'andy': 230, 'must': 2950, 'guy': 2000, 'foul': 1810, 'project': 3427, 'crew': 1053, 'friend': 1835, 'whom': 4875, 'producer': 3415, 'forget': 1793, 'commercials': 903, 'yourself': 4993, 'surprised': 4313, 'grinch': 1975, 'angels': 234, 'day': 1129, 'earned': 1401, 'glorious': 1921, 'wonderful': 4928, 'eat': 1411, 'heart': 2064, 'years': 4979, 'robert': 3713, 'deserves': 1201, 'oscar': 3133, 'production': 3419, 'storyline': 4208, 'promising': 3433, 'dutch': 1389, 'television': 4394, 'star': 4150, 'her': 2085, 'flawless': 1751, 'personally': 3243, 'she': 3923, 'professional': 3421, 'herself': 2092, 'contains': 964, 'unnecessary': 4672, 'sexual': 3908, 'shows': 3960, 'bother': 548, 'waste': 4808, 'record': 3573, 'proved': 3446, 'screen': 3829, 'title': 4504, 'creatures': 1046, 'desert': 1198, 'basically': 407, 'devoted': 1231, 'somewhat': 4068, 'kind': 2476, 'hero': 2088, 'painful': 3171, 'woody': 4936, 'allen': 188, 'wannabe': 4787, 'ultra': 4629, 'dumb': 1385, 'cop': 991, 'pick': 3265, 'irritating': 2359, 'character': 758, 'girlfriend': 1910, 'cute': 1090, 'put': 3480, 'filled': 1715, 'references': 3583, 'linda': 2609, 'blair': 497, 'already': 197, 'dated': 1119, 'early': 1400, '90s': 61, 'sucks': 4271, 'machine': 2693, 'capable': 672, 'energy': 1478, 'intriguing': 2335, 'premise': 3370, 'instead': 2307, 'happen': 2026, 'break': 569, 'damn': 1100, 'living': 2629, 'dealing': 1136, 'constant': 957, 'ie': 2221, 'invisible': 2344, 'wants': 4791, 'leave': 2555, 'doctor': 1304, 'jerry': 2396, 'himself': 2111, 'beautiful': 428, 'secretary': 3850, 'wouldnt': 4958, 'happened': 2027, 'hadnt': 2004, 'decided': 1149, 'country': 1014, 'side': 3965, 'killing': 2472, 'experiment': 1583, 'failed': 1626, 'throw': 4479, 'dry': 1377, 'british': 589, 'between': 475, 'semi': 3874, 'competent': 917, 'hired': 2117, 'loving': 2675, 'couple': 1016, 'half': 2007, 'hearted': 2065, 'attempts': 343, 'find': 1726, 'kills': 2474, 'off': 3084, 'london': 2641, 'breaking': 570, 'left': 2561, 'effort': 1432, 'broke': 593, 'making': 2718, 'again': 146, 'having': 2054, 'greek': 1970, 'suspicious': 4332, 'against': 147, 'without': 4913, 'claim': 828, 'screenplay': 3831, 'photography': 3260, 'perfect': 3224, 'technical': 4381, 'happening': 2028, 'rural': 3758, 'areas': 287, 'places': 3284, 'forgotten': 1797, 'god': 1926, 'punch': 3466, 'stomach': 4196, 'obviously': 3072, 'talks': 4361, 'anything': 260, 'different': 1248, 'problem': 3408, 'ex': 1549, 'doing': 1312, 'turn': 4605, 'speaks': 4096, 'king': 2479, 'opinion': 3118, 'religious': 3608, 'act': 97, 'says': 3801, 'same': 3778, 'easily': 1406, 'feelings': 1686, 'beliefs': 452, 'according': 88, 'established': 1524, 'system': 4345, 'positive': 3346, 'lesson': 2577, 'meaning': 2780, 'understand': 4643, 'become': 433, 'remain': 3609, 'alone': 194, 'reality': 3551, 'tragic': 4552, 'bell': 458, 'comment': 898, 'actor': 103, 'award': 366, 'small': 4033, 'plays': 3301, 'roles': 3727, 'role': 3726, 'easy': 1410, 'definitely': 1162, 'recommend': 3571, 'anybody': 257, 'need': 2987, 'arent': 288, 'beginning': 443, 'season': 3843, 'stale': 4141, 'happens': 2029, 'goes': 1930, 'ruin': 3750, 'usually': 4700, 'ending': 1471, 'guys': 2001, 'wanted': 4789, 'jeff': 2390, 'later': 2525, 'changed': 750, 'chose': 804, 'seasons': 3844, 'biggest': 480, 'surprise': 4312, 'top': 4525, 'needed': 2988, 'happy': 2032, 'glad': 1916, 'girl': 1909, 'huge': 2185, 'backdrop': 379, 'shes': 3930, 'several': 3905, 'sons': 4073, 'unhappy': 4656, 'homosexual': 2145, 'fired': 1735, 'radio': 3504, 'home': 2142, 'attend': 344, 'historical': 2119, 'develop': 1222, 'enjoyable': 1485, 'spite': 4117, 'differences': 1247, 'meeting': 2794, 'hitler': 2124, 'along': 195, 'others': 3136, 'count': 1010, 'victor': 4731, 'iii': 2227, 'events': 1536, 'voice': 4761, 'sometimes': 4067, 'romantic': 3733, 'drama': 1346, 'carried': 698, 'sense': 3877, 'passionate': 3206, 'romance': 3732, 'magnificent': 2705, 'splendid': 4119, 'course': 1019, 'results': 3662, 'elaborate': 1439, 'sentimental': 3882, 'colorful': 878, 'atmospheric': 334, 'de': 1131, 'musical': 2947, 'score': 3822, 'sensitive': 3879, 'won': 4925, 'golden': 1934, 'foreign': 1790, 'imagination': 2236, 'light': 2595, 'limited': 2606, 'where': 4864, 'developing': 1224, 'theatrical': 4430, 'example': 1553, 'le': 2544, 'uses': 4697, 'french': 1830, 'changes': 751, 'scenario': 3808, 'meet': 2793, 'unlikely': 4671, 'group': 1980, 'thomas': 4458, 'marie': 2737, 'revolutionary': 3681, 'paris': 3189, 'family': 1645, 'familys': 1646, 'grand': 1953, 'roman': 3731, 'flat': 1748, 'strikes': 4225, 'marvelous': 2750, '20': 29, 'ago': 154, 'beautifully': 429, 'filmed': 1717, 'keeps': 2451, 'entire': 1502, 'difficult': 1249, 'fresh': 1833, 'today': 4508, 'regarding': 3589, 'technique': 4383, 'terrific': 4409, 'considering': 952, 'addition': 118, 'superb': 4296, 'english': 1483, 'american': 213, 'purchased': 3470, 'able': 66, 'whenever': 4863, 'anyone': 259, 'post': 3352, 'yes': 4982, 'slightly': 4029, 'hours': 2177, 'masterpieces': 2760, 'cinema': 816, 'savage': 3793, 'soviet': 4089, 'type': 4621, 'under': 4639, 'faithful': 1634, 'adaptation': 113, 'banned': 395, 'sets': 3899, 'decision': 1151, 'adding': 117, 'late': 2523, '80s': 59, 'color': 877, 'ruined': 3751, 'wont': 4932, 'discovered': 1280, 'viewer': 4738, 'material': 2766, 'mentions': 2813, 'professor': 3422, 'presence': 3373, 'wisdom': 4902, 'ends': 1474, 'stealing': 4171, 'creature': 1045, 'horrifying': 2167, 'perfection': 3225, 'frightening': 1839, 'exist': 1569, 'countries': 1013, 'ran': 3513, 'men': 2805, 'throughout': 4478, 'history': 2121, 'ensues': 1493, 'political': 3324, 'mentioning': 2812, 'humor': 2192, 'albeit': 173, 'dark': 1115, 'laughing': 2533, 'feeling': 1685, 'hardly': 2036, 'complaint': 920, 'slight': 4027, 'flaw': 1749, 'tone': 4517, 'intellectual': 2312, 'caught': 722, 'minor': 2857, 'sadly': 3769, 'appears': 278, 'western': 4853, 'shame': 3918, 'round': 3742, 'incredibly': 2270, 'cultural': 1078, 'necessarily': 2983, 'translation': 4562, 'themes': 4434, 'waiting': 4772, 'edition': 1422, 'creating': 1039, 'works': 4945, 'nick': 3012, 'cage': 645, 'treasure': 4571, 'hunter': 2199, 'oh': 3096, 'excuse': 1564, 'means': 2783, 'clue': 864, 'fathers': 1667, 'hide': 2097, 'badly': 383, 'turns': 4609, 'ship': 3934, 'hidden': 2096, 'ian': 2209, 'sean': 3840, 'justin': 2440, 'enter': 1494, 'hold': 2129, 'immediately': 2242, 'tons': 4520, 'floor': 1759, 'significant': 3972, 'sort': 4077, 'ask': 316, 'explain': 1586, 'despite': 1211, 'finding': 1727, 'actual': 108, 'around': 297, 'stands': 4147, 'disappointment': 1275, 'coming': 896, 'enters': 1496, 'steal': 4170, 'various': 4715, 'leads': 2548, 'pulling': 3463, 'gun': 1996, 'explosion': 1596, 'huh': 2187, 'escape': 1517, 'walk': 4775, 'nine': 3017, 'miles': 2843, 'village': 4743, 'order': 3126, 'fbi': 1675, 'super': 4295, 'diane': 1238, 'tells': 4397, 'detail': 1217, 'metal': 2823, 'eyes': 1613, 'security': 3854, 'blah': 496, 'opportunity': 3121, 'room': 3735, 'asks': 319, 'kidding': 2465, 'joke': 2414, 'either': 1438, 'research': 3644, 'library': 2589, 'water': 4816, 'level': 2584, 'script': 3834, 'short': 3946, 'ridiculous': 3693, 'hate': 2046, 'computer': 927, 'technology': 4385, 'graphics': 1959, 'sake': 3775, 'interest': 2323, 'angle': 236, 'point': 3316, 'three': 4469, 'run': 3755, 'law': 2538, 'discuss': 1283, 'plans': 3292, 'clothing': 861, 'store': 4204, 'surrounded': 4318, 'clues': 865, 'father': 1666, 'jon': 2416, 'voight': 4764, 'alike': 185, 'jones': 2418, 'house': 2178, 'odd': 3080, 'wild': 4885, 'believes': 456, 'trapped': 4564, 'leaves': 2556, 'gone': 1935, 'boy': 557, 'button': 639, 'door': 1328, 'opens': 3115, 'serious': 3891, 'office': 3091, 'building': 615, 'knew': 2486, 'anyway': 261, 'sorts': 4078, 'stupid': 4247, 'officer': 3092, 'harvey': 2042, 'allows': 192, 'chemistry': 782, 'less': 2575, 'married': 2744, 'logic': 2638, 'involves': 2348, 'lessons': 2578, 'conversation': 980, 'random': 3514, 'moments': 2887, 'id': 2212, 'conversations': 981, 'note': 3044, 'dramatic': 1348, 'wanting': 4790, 'sequel': 3885, 'mention': 2810, 'depressing': 1188, 'perry': 3237, 'harder': 2035, 'kinda': 2477, 'dull': 1384, '10': 0, 'jay': 2386, 'spanish': 4092, 'each': 1395, 'greatly': 1967, 'disappointed': 1273, 'canadian': 667, 'parents': 3188, 'industry': 2280, 'further': 1853, 'wasnt': 4807, 'melodrama': 2797, 'types': 4622, 'alas': 172, 'ego': 1435, 'trip': 4588, 'japanese': 2382, 'depth': 1190, 'complexity': 924, 'dimensional': 1253, 'slow': 4031, 'historically': 2120, 'philip': 3252, 'anthony': 253, 'etc': 1527, 'mainstream': 2709, 'attracted': 349, 'audiences': 353, 'meant': 2784, 'quality': 3484, 'information': 2288, 'awards': 367, 'foot': 1781, 'afraid': 140, 'answer': 251, 'ludicrous': 2685, 'nowadays': 3055, 'likely': 2601, 'straight': 4210, 'non': 3027, 'calling': 653, 'heavily': 2069, 'hitchcock': 2123, 'pure': 3471, 'images': 2235, 'words': 4938, 'convey': 982, 'definition': 1163, 'bit': 488, 'mood': 2898, 'simply': 3986, 'landscape': 2513, 'featuring': 1681, 'dying': 1393, 'mother': 2911, 'grandmother': 1955, 'spoilers': 4124, 'brooklyn': 595, 'north': 3036, 'eddie': 1416, 'drug': 1373, 'larry': 2520, 'section': 3853, 'operation': 3117, 'during': 1387, 'deadly': 1133, 'innocent': 2294, 'six': 4010, 'year': 4978, 'bone': 533, 'fire': 1734, 'host': 2173, '1996': 26, 'summer': 4291, 'young': 4989, 'determined': 1221, 'bullet': 620, 'member': 2800, 'police': 3322, 'department': 1182, 'shocking': 3940, 'killer': 2470, 'state': 4160, 'judge': 2424, 'walter': 4784, 'arrested': 298, 'soon': 4074, 'became': 431, 'evident': 1547, 'job': 2405, 'frank': 1821, 'land': 2511, 'involving': 2349, 'bring': 585, 'estate': 1525, 'friends': 1837, 'millions': 2848, 'dollars': 1315, 'connect': 944, 'free': 1827, 'keep': 2449, 'blowing': 515, 'working': 4944, 'authority': 360, 'hiding': 2099, 'cash': 709, 'uncle': 4636, 'mafia': 2700, 'paul': 3214, 'letting': 2583, 'drugs': 1374, 'piece': 3272, 'paying': 3217, 'kevin': 2457, 'criminal': 1057, 'allowed': 190, 'streets': 4217, 'major': 2711, 'turned': 4606, 'connected': 945, 'hip': 2114, 'corruption': 1002, 'knowing': 2490, 'sin': 3988, 'jobs': 2406, 'took': 4523, 'motion': 2913, 'besides': 468, 'street': 4216, 'popular': 3333, 'looking': 2648, 'forward': 1808, 'bigger': 479, 'president': 3378, 'future': 1856, 'disaster': 1276, 'dozen': 1338, 'murders': 2943, 'suicide': 4283, 'evil': 1548, 'chance': 747, 'double': 1331, 'combination': 883, 'omen': 3104, 'seed': 3856, 'deceased': 1146, 'neighbors': 2994, 'named': 2961, 'karen': 2444, 'gene': 1879, 'fay': 1674, 'grant': 1956, 'michael': 2829, 'catholic': 720, 'st': 4137, 'francis': 1818, 'mom': 2885, 'party': 3200, 'truck': 4595, 'devil': 1228, 'dog': 1310, 'school': 3813, 'care': 684, 'local': 2634, 'wet': 4855, 'front': 1841, 'suit': 4284, 'head': 2057, 'accident': 84, 'mess': 2818, 'meanwhile': 2785, 'becomes': 434, 'clean': 838, 'trees': 4577, 'crowd': 1067, 'boys': 559, 'neighborhood': 2993, 'eye': 1611, 'itself': 2371, 'strange': 4211, 'daughter': 1121, 'sudden': 4272, 'fortune': 1806, 'ann': 244, 'powers': 3361, 'age': 148, 'realizes': 3554, 'weird': 4841, 'turning': 4608, 'calls': 654, 'jim': 2402, 'check': 777, 'upset': 4687, 'sees': 3866, 'blue': 518, 'taken': 4350, 'psychic': 3456, 'festival': 1697, 'setting': 3900, 'mental': 2808, 'ended': 1470, 'leg': 2562, 'experience': 1580, 'thrown': 4481, 'help': 2079, 'pet': 3246, 'window': 4894, 'talked': 4359, 'pregnant': 3369, 'truth': 4599, 'face': 1616, 'twin': 4615, 'sister': 4001, 'brother': 598, 'alexander': 180, 'enough': 1491, 'earth': 1403, 'public': 3460, 'adventures': 131, 'fourth': 1813, 'necessary': 2984, 'since': 3990, 'dead': 1132, 'buried': 624, 'usual': 4699, 'believing': 457, 'destroying': 1215, 'race': 3499, 'minutes': 2859, 'horror': 2168, 'flick': 1754, '80': 58, 'earl': 1398, 'former': 1800, 'faith': 1633, 'suffering': 4277, 'began': 441, 'directed': 1259, 'acted': 98, 'belief': 451, 'casting': 711, 'service': 3896, 'jennifer': 2392, 'polished': 3323, 'believable': 453, 'richard': 3686, 'miscast': 2862, 'particular': 3194, 'peter': 3247, 'jenny': 2393, 'awful': 372, 'slave': 4022, 'keeping': 2450, 'although': 201, 'normally': 3034, 'fine': 1729, 'attempted': 341, 'accents': 79, 'african': 142, 'thus': 4486, 'features': 1680, 'blond': 510, 'bothered': 549, 'appropriate': 283, 'large': 2517, 'pool': 3327, 'extremely': 1610, 'talented': 4355, 'physically': 3263, 'respect': 3652, 'choose': 799, 'daniel': 1111, 'direction': 1261, 'robbins': 3712, 'books': 536, 'capture': 676, 'magic': 2703, 'blues': 519, 'include': 2261, 'content': 966, 'focus': 1766, 'connection': 946, 'else': 1448, 'weak': 4824, 'five': 1743, 'minute': 2858, 'clip': 851, 'saturday': 3792, 'rent': 3626, 'forced': 1787, 'hour': 2176, 'hilarious': 2106, 'rather': 3528, 'tame': 4363, 'concerning': 933, 'ii': 2226, 'stolen': 4195, 'expected': 1577, 'mgm': 2828, 'jean': 2389, 'rogers': 3725, 'flash': 1745, 'gordon': 1940, 'overacting': 3149, 'secret': 3849, 'code': 869, 'official': 3094, 'watched': 4813, 'forever': 1792, 'ms': 2931, 'extreme': 1609, 'merely': 2815, 'adequate': 120, 'pushing': 3479, 'lovely': 2671, 'fairy': 1632, 'detailed': 1218, 'names': 2963, 'dialogue': 1234, 'deep': 1154, 'sky': 4017, 'natural': 2973, 'satisfying': 3791, 'song': 4071, 'gorgeous': 1942, 'likable': 2598, 'leading': 2547, 'touch': 4533, 'shell': 3927, 'die': 1243, 'tension': 4404, 'wished': 4905, 'la': 2500, 'pleasure': 3306, 'spend': 4105, 'hasnt': 2044, 'crappy': 1032, 'week': 4837, 'company': 910, 'eager': 1396, 'featured': 1679, 'favorite': 1671, 'soldiers': 4054, 'border': 539, 'holes': 2133, 'revealed': 3671, 'journalist': 2420, 'woman': 4921, '15': 7, 'apart': 264, 'lady': 2507, 'telling': 4396, 'shown': 3959, 'moment': 2886, 'bar': 396, 'corner': 996, 'winter': 4900, 'begins': 444, 'continuity': 972, 'problems': 3409, 'absurd': 73, 'asking': 318, 'questions': 3489, 'answers': 252, 'lacked': 2503, 'integrity': 2311, 'charm': 767, 'showed': 3956, 'politically': 3325, 'humanity': 2189, 'lose': 2654, 'realise': 3548, 'cult': 1077, 'franco': 1820, 'surreal': 4317, 'reputation': 3640, 'unfortunately': 4654, 'truly': 4597, 'admit': 122, 'cut': 1089, 'runs': 3757, 'unless': 4669, 'longer': 2645, 'european': 1530, 'rubbish': 3748, 'gore': 1941, 'trash': 4565, 'describe': 1194, 'choices': 798, 'blank': 501, 'liked': 2600, 'russian': 3762, 'sticks': 4186, 'pants': 3182, 'hits': 2125, 'twist': 4617, 'choice': 797, 'bored': 541, 'aint': 163, 'joan': 2404, 'ben': 463, 'sent': 3880, 'middle': 2833, 'eastern': 1408, 'oil': 3097, 'wedding': 4836, 'falls': 1640, 'news': 3005, 'afterwards': 145, 'throws': 4482, 'reveals': 3673, 'entertaining': 1499, 'elephant': 1444, 'inspired': 2301, 'recent': 3566, 'wave': 4818, 'unlike': 4670, 'focuses': 1768, 'survivors': 4325, 'useful': 4694, 'wasted': 4809, 'hill': 2108, 'episode': 1507, 'lame': 2510, 'bear': 420, 'running': 3756, 'length': 2569, 'entirely': 1503, 'jumping': 2431, 'ahead': 159, 'hospital': 2172, 'ryan': 3765, 'guarantee': 1987, 'sign': 3970, 'certain': 737, 'construction': 960, 'execution': 1566, 'often': 3095, 'lacking': 2504, 'budget': 610, 'strength': 4219, 'executed': 1565, 'vision': 4754, 'onto': 3111, 'limits': 2607, 'average': 362, 'edge': 1418, 'journey': 2421, 'sarah': 3785, 'amazing': 209, 'editing': 1421, 'final': 1722, 'busy': 635, 'protagonists': 3442, 'standard': 4144, 'teen': 4388, 'intelligent': 2314, 'intensity': 2317, 'detective': 1220, 'wondering': 4930, 'tie': 4488, 'lone': 2642, 'helping': 2081, 'novel': 3052, 'fascinating': 1658, 'convincing': 985, 'slowly': 4032, 'tears': 4380, 'progress': 3425, 'expecting': 1578, 'ultimately': 4628, 'suspense': 4330, 'actresses': 106, 'brought': 600, 'child': 789, 'listen': 2619, 'jokes': 2415, 'swear': 4334, 'asian': 314, 'wouldve': 4959, 'funnier': 1850, 'crap': 1031, 'ill': 2228, 'claimed': 829, 'stopped': 4202, 'taking': 4352, 'apparently': 270, 'mostly': 2910, 'related': 3594, 'crack': 1027, 'ass': 323, 'shut': 3961, 'thank': 4421, 'youre': 4992, 'notorious': 3051, 'prize': 3405, 'winning': 4898, 'simon': 3982, 'grim': 1974, 'loves': 2674, 'henry': 2084, 'fool': 1779, 'receives': 3565, 'box': 555, 'collection': 874, 'agents': 151, 'terrorists': 4414, 'flight': 1757, 'parker': 3191, 'delivers': 1172, 'perfectly': 3226, 'comic': 893, 'physical': 3262, 'strong': 4229, 'proves': 3447, 'tradition': 4549, 'trouble': 4592, 'academy': 77, 'berlin': 467, 'served': 3894, 'fellow': 1691, '2004': 34, 'santa': 3783, 'jaw': 2384, 'ways': 4822, 'husband': 2204, 'holy': 2140, 'points': 3319, 'pig': 3274, 'olivier': 3103, 'discovers': 1281, 'target': 4366, 'wears': 4832, 'sight': 3969, 'theater': 4427, 'release': 3602, 'ah': 158, 'spider': 4109, 'guess': 1989, 'co': 867, 'productions': 3420, 'italian': 2367, 'ha': 2002, 'thrillers': 4473, 'mysteries': 2954, 'scripted': 3835, 'screenwriter': 3833, 'stick': 4185, 'del': 1165, 'devils': 1229, 'clever': 841, 'gay': 1876, 'twins': 4616, 'dating': 1120, 'stuff': 4243, 'needless': 2989, 'offensive': 3086, 'remarks': 3615, 'apparent': 269, 'painting': 3175, 'revelation': 3674, 'finish': 1732, 'reveal': 3670, 'directing': 1260, 'knowledge': 2491, 'change': 749, 'pace': 3161, 'giant': 1905, 'bore': 540, 'rule': 3753, 'saying': 3800, 'mystery': 2956, 'killers': 2471, 'mighty': 2836, 'existed': 1570, 'producers': 3416, 'speech': 4102, 'nor': 3032, 'pay': 3216, 'greatest': 1966, 'actress': 105, 'current': 1084, 'attraction': 350, 'lines': 2612, 'uninteresting': 4660, 'shallow': 3917, 'incoherent': 2265, 'implausible': 2245, 'wow': 4962, 'petty': 3249, 'selfish': 3870, 'bizarre': 492, 'widow': 4883, 'starts': 4159, 'daughters': 1122, 'boyfriend': 558, 'utterly': 4702, 'anne': 246, 'craig': 1030, 'latter': 2528, 'roger': 3724, 'compelled': 915, 'write': 4967, 'impossible': 2249, 'sequences': 3888, 'willing': 4889, 'ring': 3698, 'judy': 2426, 'lees': 2560, 'involvement': 2347, 'deeper': 1155, 'mob': 2879, 'gang': 1867, 'thugs': 4484, 'dubbing': 1380, 'kung': 2498, 'cheesy': 781, 'sequence': 3887, 'familiar': 1643, 'pulp': 3465, 'fiction': 1702, 'continues': 971, 'square': 4136, 'rating': 3529, 'cared': 685, 'hands': 2020, 'approach': 282, '17': 9, 'becoming': 435, 'dancing': 1107, 'lots': 2662, 'low': 2676, 'wife': 4884, 'seek': 3858, 'picks': 3268, 'poorly': 3329, 'grow': 1982, 'caring': 691, 'attempt': 340, 'utter': 4701, 'typical': 4623, 'exception': 1557, 'narrative': 2966, 'contrived': 976, 'context': 968, 'britain': 588, 'tiresome': 4502, 'view': 4736, 'poverty': 3357, 'margaret': 2735, 'classes': 834, 'ordinary': 3128, 'clips': 852, 'added': 116, 'randomly': 3515, 'towards': 4540, 'effect': 1428, 'tad': 4347, 'disjointed': 1287, 'moved': 2922, 'couldve': 1009, 'quickly': 3491, 'couldnt': 1008, 'younger': 4990, 'looked': 2647, 'four': 1812, 'reach': 3536, 'difference': 1246, 'teens': 4392, 'occurs': 3078, 'mature': 2773, 'older': 3101, 'created': 1037, 'shock': 3938, 'throwing': 4480, 'incident': 2260, 'fell': 1690, 'bugs': 613, 'concern': 931, 'upper': 4685, 'hung': 2197, 'class': 833, 'born': 544, '1973': 19, '1983': 22, 'grew': 1972, 'single': 3996, 'parent': 3187, 'england': 1482, 'childhood': 790, 'experiences': 1582, 'portrayed': 3342, 'racism': 3502, 'issue': 2364, 'bottom': 551, 'substance': 4258, 'missed': 2867, 'texas': 4418, '50s': 52, 'revolves': 3682, 'students': 4239, 'adults': 126, 'teenagers': 4391, 'football': 1783, 'johnson': 2411, 'dies': 1245, 'timothy': 4499, 'joe': 2407, 'bob': 524, 'pathetic': 3210, 'body': 527, 'intended': 2315, 'suppose': 4305, 'grey': 1973, 'aside': 315, 'terrifying': 4410, 'cousin': 1021, 'national': 2970, 'vacation': 4704, 'showing': 3958, 'witnesses': 4916, 'families': 1644, 'generation': 1883, 'appreciated': 280, 'witness': 4914, 'members': 2801, 'accept': 80, 'visits': 4756, 'medical': 2790, 'allow': 189, 'sloppy': 4030, 'whereas': 4865, 'causes': 725, 'trained': 4557, 'prefer': 3368, 'treat': 4572, 'loved': 2670, 'raised': 3508, 'stayed': 4167, 'sadness': 3770, 'drivel': 1364, 'puts': 3481, 'gotten': 1947, '25': 40, 'builds': 617, 'drawn': 1351, 'horrid': 2165, 'wooden': 4934, 'predictable': 3367, 'absolute': 71, 'jessica': 2398, 'women': 4923, 'portraying': 3343, 'max': 2774, 'mr': 2929, 'presentation': 3375, 'unfortunate': 4653, 'portrayal': 3341, 'supporting': 4304, 'amanda': 205, 'hugh': 2186, 'mel': 2796, 'consequences': 947, 'game': 1864, 'remembered': 3617, 'bag': 384, 'captures': 678, 'nostalgia': 3038, 'wound': 4960, 'effective': 1429, 'follow': 1772, 'shark': 3921, 'international': 2326, 'psychotic': 3459, 'complex': 923, 'psycho': 3457, 'fatal': 1664, 'mixed': 2877, 'object': 3066, 'career': 686, 'levels': 2585, 'complicated': 925, 'finest': 1730, 'roy': 3745, 'glover': 1923, 'notable': 3041, 'unexpected': 4650, 'kelly': 2452, 'likes': 2602, 'combined': 885, 'gritty': 1977, 'underrated': 4642, 'bill': 481, 'adventure': 130, 'ted': 4386, 'aka': 168, 'fetched': 1698, 'juvenile': 2441, 'brilliantly': 584, 'idiots': 2220, 'situation': 4008, 'brains': 563, 'willis': 4890, 'arnold': 296, 'blockbuster': 509, 'balance': 387, 'business': 633, 'case': 707, 'mistakes': 2873, 'china': 795, 'student': 4238, 'dialog': 1232, 'amounts': 220, 'violence': 4747, 'lack': 2502, 'development': 1225, 'reporter': 3636, 'plant': 3293, 'causing': 726, 'ruthless': 3764, 'chased': 770, 'evidence': 1546, 'faults': 1669, 'idiotic': 2219, 'appeal': 271, 'endure': 1475, 'violent': 4748, 'filmmakers': 1720, 'focused': 1767, 'greater': 1965, 'attention': 345, 'aspects': 322, 'creepy': 1052, 'villains': 4745, 'naive': 2958, 'alert': 178, 'formulaic': 1803, 'memories': 2803, 'lived': 2626, 'sunday': 4293, 'evening': 1534, '14': 6, 'provides': 3450, 'voices': 4763, 'nazi': 2977, 'von': 4765, 'enemy': 1477, 'disappeared': 1271, 'escaped': 1518, 'leaving': 2557, 'appear': 273, 'germany': 1898, 'obsession': 3070, 'revenge': 3675, 'delight': 1167, 'youth': 4994, 'hunt': 2198, 'normal': 3033, 'standing': 4146, 'memory': 2804, 'prisoner': 3402, 'copies': 992, 'hopefully': 2157, 'canada': 666, 'finished': 1733, 'dreadful': 1353, 'nude': 3058, 'lol': 2640, 'sick': 3963, 'annoyed': 248, 'loud': 2664, 'screams': 3828, 'qualities': 3483, 'please': 3304, 'consider': 949, 'wanna': 4786, 'versions': 4721, 'perhaps': 3235, 'copy': 994, '30': 42, 'florida': 1761, 'extent': 1605, 'succeeded': 4263, 'pretentious': 3383, 'within': 4912, 'near': 2979, 'killings': 2473, 'concerned': 932, 'explained': 1587, 'silly': 3976, 'common': 906, 'betty': 474, 'blonde': 511, 'amazed': 208, 'jump': 2429, 'seat': 3845, 'art': 304, '12': 3, 'drop': 1369, 'glass': 1917, 'orange': 3125, 'guest': 1992, 'reviewers': 3678, 'create': 1036, 'costumes': 1006, 'closely': 855, 'largely': 2518, 'personalities': 3241, 'sitting': 4007, 'taxi': 4371, 'driver': 1366, 'walked': 4776, 'date': 1118, 'cheap': 774, 'brain': 562, 'represents': 3639, 'smart': 4035, 'sophisticated': 4075, 'era': 1513, 'laughs': 2534, 'honestly': 2147, 'indeed': 2271, 'awesome': 371, 'concept': 930, 'realism': 3549, 'study': 4242, 'nature': 2975, 'parallel': 3185, 'marks': 2742, 'product': 3418, 'tarzan': 4367, 'sounded': 4082, 'ape': 266, 'hundred': 2195, 'drive': 1363, 'remarkable': 3613, 'tend': 4400, 'expensive': 1579, 'developed': 1223, 'jungle': 2434, 'criticism': 1063, 'praise': 3364, 'let': 2579, 'kiss': 2483, 'murder': 2939, 'carry': 700, 'river': 3707, 'proof': 3434, 'granted': 1957, 'forgot': 1796, 'tough': 4537, 'kill': 2468, 'floating': 1758, 'seconds': 3848, 'picking': 3267, 'apes': 267, 'brings': 587, 'punishment': 3467, 'nonsense': 3030, 'struggle': 4234, 'slap': 4018, 'kick': 2460, 'americas': 215, 'titled': 4505, 'americans': 214, 'downright': 1337, 'disgusting': 1286, 'dave': 1123, 'save': 3794, 'neither': 2996, 'segments': 3868, 'hell': 2077, 'videos': 4734, 'variety': 4714, 'prime': 3395, 'started': 4157, 'winner': 4897, 'mentioned': 2811, 'theme': 4433, 'rocks': 3722, 'recognized': 3570, 'treatment': 4574, 'built': 618, 'bought': 552, 'jack': 2373, 'george': 1893, 'washington': 4806, 'hart': 2041, 'fifteen': 1706, 'hanks': 2025, 'pit': 3278, 'cary': 706, 'tight': 4492, 'apartment': 265, 'share': 3920, 'advertising': 132, 'executive': 1567, 'feels': 1687, 'financial': 1725, 'hole': 2132, 'buying': 641, 'ho': 2127, 'fail': 1625, 'warning': 4800, 'wise': 4903, 'cynical': 1093, 'lawyer': 2541, 'cole': 873, 'douglas': 1334, 'nicely': 3009, 'checking': 778, 'correct': 1000, 'cost': 1003, 'thousand': 4465, 'soft': 4051, 'designs': 1204, 'dropped': 1370, 'build': 614, 'planned': 3290, 'costs': 1004, 'handle': 2018, 'account': 89, 'result': 3660, 'armed': 293, 'forces': 1788, 'following': 1774, 'poetry': 3314, 'pink': 3277, 'ham': 2011, 'increasingly': 2268, 'locked': 2637, 'constructed': 959, 'open': 3112, 'panic': 3181, 'tries': 4585, 'breaks': 571, 'enjoying': 1487, 'price': 3389, 'paint': 3173, 'delightful': 1168, 'rooms': 3736, 'carefully': 689, 'distinct': 1296, 'giving': 1915, 'wide': 4880, 'colors': 879, 'mrs': 2930, 'arthur': 305, 'queen': 3485, 'leonard': 2572, 'wait': 4770, 'lovers': 2673, 'horrific': 2166, 'edgar': 1417, 'quiet': 3492, 'staged': 4140, 'okay': 3099, 'rights': 3697, 'produce': 3413, 'brand': 565, 'ninja': 3018, 'naturally': 2974, 'managed': 2723, 'threw': 4470, 'tends': 4402, 'darker': 1116, 'terms': 4406, 'suits': 4287, 'complain': 919, 'rid': 3690, 'rangers': 3517, 'beat': 423, 'countless': 1012, 'eventually': 1537, 'encounters': 1467, 'stated': 4161, 'female': 1693, 'infamous': 2284, 'comfortable': 892, 'impression': 2252, 'enemies': 1476, 'fortunately': 1805, 'creators': 1044, 'flawed': 1750, 'episodes': 1508, 'explanation': 1590, 'warned': 4798, 'appreciate': 279, 'neil': 2995, 'wit': 4908, 'ear': 1397, 'jewish': 2401, 'expression': 1602, 'heres': 2087, 'matthau': 2771, 'refuses': 3587, 'attitudes': 347, 'vulnerable': 4768, 'scared': 3805, 'husbands': 2205, 'roots': 3738, 'method': 2824, 'delivery': 1173, 'nicholson': 3011, 'competition': 918, 'separate': 3883, 'oscars': 3134, 'afternoon': 144, 'sunshine': 4294, 'masterful': 2758, 'legendary': 2565, 'burns': 628, 'reasons': 3560, 'return': 3665, 'chris': 806, 'stunning': 4244, 'theory': 4437, 'expressions': 1603, 'seemingly': 3863, 'currently': 1085, 'pain': 3170, 'relief': 3605, 'witty': 4917, 'humour': 2194, 'matt': 2768, 'bits': 490, 'entertained': 1498, 'humorous': 2193, 'quotes': 3497, 'ps': 3453, 'heroine': 2091, 'empathy': 1462, 'individual': 2277, 'personality': 3242, 'generous': 1886, 'producing': 3417, 'todays': 4509, 'anti': 254, 'redeeming': 3577, 'aid': 160, 'quest': 3486, 'wind': 4893, 'blows': 517, 'friday': 1834, 'morning': 2906, 'fill': 1714, 'purely': 3472, 'track': 4544, 'techniques': 4384, 'lately': 2524, 'seventies': 3904, 'subtle': 4260, 'touches': 4535, 'manages': 2725, 'laughter': 2535, 'trek': 4578, 'bloody': 513, 'opened': 3113, 'pilot': 3276, 'captain': 674, 'refreshing': 3586, 'mirror': 2861, 'considered': 951, 'tell': 4395, 'odds': 3082, 'conventional': 979, 'rush': 3759, 'spot': 4131, 'stereotypes': 4179, 'anger': 235, 'keaton': 2448, 'parody': 3192, 'cagney': 646, 'spare': 4093, 'solid': 4057, 'workers': 4943, 'step': 4175, 'spoof': 4127, '30s': 44, '40s': 49, 'gangster': 1868, 'remarkably': 3614, 'yeah': 4977, 'elsewhere': 1449, 'warn': 4797, 'teacher': 4375, 'gags': 1862, 'owner': 3159, 'johnny': 2410, 'tag': 4348, 'educational': 1425, 'church': 813, 'latin': 2527, 'shop': 3945, 'decade': 1144, 'patient': 3211, 'rolling': 3730, 'viewings': 4741, 'flaws': 1752, 'none': 3028, 'wealthy': 4827, 'smoking': 4040, 'period': 3236, 'welcome': 4842, 'offers': 3090, 'reminded': 3619, 'discover': 1279, 'madonna': 2699, 'mario': 2738, 'portray': 3340, 'william': 4887, 'macy': 2695, 'ensemble': 1492, 'shine': 3931, 'presents': 3377, 'inane': 2258, 'safe': 3771, 'succeed': 4262, 'master': 2757, 'confused': 941, 'priest': 3392, 'funeral': 1849, 'doomed': 1327, 'lowest': 2678, 'rated': 3527, 'wins': 4899, 'dick': 1239, 'tracy': 4546, 'scripts': 3836, 'holmes': 2139, 'lord': 2652, 'albert': 174, 'earlier': 1399, 'scientific': 3817, 'wondered': 4927, 'aired': 165, 'tape': 4365, 'animation': 242, 'entertain': 1497, 'mentally': 2809, 'retarded': 3663, 'insult': 2309, 'adult': 125, 'amusing': 221, 'murdered': 2940, 'inept': 2281, 'empire': 1464, 'stood': 4199, 'dirty': 1268, 'upon': 4684, 'ancient': 225, 'vhs': 4725, 'obscure': 3068, 'pleasant': 3302, 'ages': 152, 'present': 3374, 'attacks': 339, 'museum': 2945, 'accused': 91, 'twenty': 4612, 'led': 2558, 'attacked': 338, 'burned': 626, 'unfolds': 4651, 'mixture': 2878, 'measure': 2786, 'senseless': 3878, 'boredom': 542, 'contemporary': 965, 'terror': 4412, 'forth': 1804, 'flashbacks': 1747, 'fast': 1661, 'paced': 3162, 'reminiscent': 3621, 'williams': 4888, 'known': 2492, 'stage': 4139, 'depicted': 1183, 'committed': 905, 'ghost': 1902, 'directorial': 1265, 'goldberg': 1933, 'crime': 1055, 'ice': 2210, 'search': 3841, 'jonathan': 2417, 'mere': 2814, 'fictional': 1703, 'soap': 4047, 'opera': 3116, 'plots': 3310, 'sub': 4252, 'mine': 2853, 'christopher': 811, 'arrives': 302, 'brown': 601, 'werewolf': 4850, 'annie': 247, 'join': 2412, 'horrendous': 2162, 'sink': 3999, 'cartoon': 704, 'eight': 1436, 'closing': 859, 'universal': 4665, '1930s': 11, 'tricks': 4583, 'transition': 4561, 'bodies': 526, 'unable': 4631, 'destruction': 1216, 'fascinated': 1657, 'initially': 2291, 'somehow': 4063, 'promised': 3431, 'club': 863, 'faced': 1617, 'animal': 239, 'maker': 2714, 'climax': 849, 'display': 1291, 'visually': 4758, 'attractive': 351, 'affair': 135, 'appeared': 276, 'simplistic': 3985, 'wing': 4896, 'block': 508, 'analysis': 224, 'modern': 2882, 'ken': 2453, 'starting': 4158, 'struggling': 4236, 'successful': 4266, 'social': 4049, 'van': 4713, 'clichés': 846, '16': 8, 'move': 2921, 'vehicle': 4718, 'jeremy': 2394, 'teenager': 4390, 'baseball': 403, 'relate': 3593, 'awe': 370, 'literature': 2623, 'touched': 4534, 'tales': 4357, 'filmmaker': 1719, 'latest': 2526, 'background': 380, 'bank': 394, 'tense': 4403, 'surrounding': 4319, 'lifestyle': 2593, 'teenage': 4389, 'magazine': 2701, 'editor': 1423, 'similarly': 3980, 'false': 1641, 'politics': 3326, 'manager': 2724, 'rarely': 3524, 'engage': 1479, 'contrary': 974, 'activities': 102, 'closest': 857, 'friendly': 1836, 'prevent': 3385, 'duty': 1390, 'thoroughly': 4459, 'birth': 486, 'efforts': 1433, 'initial': 2290, 'stays': 4169, 'grown': 1984, 'accent': 78, 'newly': 3004, 'identity': 2217, 'prisoners': 3403, 'visible': 4753, 'access': 83, 'statement': 4162, 'awkward': 374, 'chain': 741, 'emotional': 1459, 'impact': 2244, 'spoken': 4126, 'flicks': 1755, 'godfather': 1927, 'league': 2549, 'dropping': 1371, 'spoiled': 4122, 'don': 1320, 'disappear': 1270, 'smiling': 4037, 'college': 875, 'cruel': 1070, 'phone': 3255, 'promise': 3430, 'fashioned': 1660, 'slasher': 4020, 'rich': 3685, 'designed': 1203, 'mainly': 2708, 'carries': 699, 'deals': 1137, 'manner': 2729, 'genuine': 1891, 'conflict': 939, 'wishes': 4906, 'arrived': 301, 'characterization': 759, 'otherwise': 3137, 'brian': 576, 'suspenseful': 4331, 'manage': 2722, 'surprisingly': 4316, 'realistic': 3550, 'emotion': 1458, 'recommended': 3572, 'thrills': 4475, 'taste': 4369, 'driven': 1365, 'unpleasant': 4674, 'folk': 1770, 'comments': 901, 'renting': 3629, 'garbage': 1870, 'rented': 3628, 'knock': 2488, 'values': 4710, 'spoiler': 4123, 'raped': 3522, 'chief': 788, 'staring': 4152, 'advise': 134, 'endearing': 1469, 'portrait': 3339, 'east': 1407, 'inner': 2292, 'cities': 823, 'losers': 2656, 'hollywood': 2137, 'fooled': 1780, 'frustration': 1843, 'welles': 4844, 'citizen': 824, 'kane': 2442, 'prove': 3445, 'scheme': 3812, 'africa': 141, 'handsome': 2021, 'monk': 2889, 'crisis': 1060, 'happiness': 2031, 'garden': 1872, 'charles': 764, 'understated': 4646, 'sympathetic': 4342, 'joseph': 2419, 'guide': 1993, 'speak': 4094, 'gentle': 1890, 'superior': 4300, 'alan': 171, 'dangerous': 1110, 'carradine': 695, 'mysterious': 2955, 'sir': 4000, 'smith': 4038, 'recognize': 3569, 'helen': 2075, 'hotel': 2175, 'rare': 3523, 'location': 2635, 'drinking': 1362, 'degree': 1164, 'heat': 2067, 'daily': 1097, 'bright': 581, 'spooky': 4128, 'string': 4227, 'restaurant': 3658, 'painfully': 3172, 'rising': 3703, 'due': 1382, 'chances': 748, 'satisfied': 3789, 'alex': 179, 'meets': 2795, 'downhill': 1336, 'circumstances': 822, 'heaven': 2068, 'similar': 3978, '90': 60, 'smile': 4036, 'suffers': 4278, 'manipulative': 2727, 'melodramatic': 2798, 'childish': 791, 'briefly': 580, 'distant': 1295, 'relative': 3599, 'lovable': 2668, 'mild': 2839, 'moves': 2925, 'dogs': 1311, 'pictures': 3270, 'endless': 1473, 'dislike': 1288, 'ray': 3533, 'firstly': 1737, 'hint': 2112, 'dub': 1378, 'exceptionally': 1559, 'jealous': 2388, 'forgive': 1795, 'hang': 2022, 'forest': 1791, 'explaining': 1588, 'humans': 2190, 'moon': 2900, 'call': 651, 'ratings': 3530, 'twice': 4613, 'slightest': 4028, 'walken': 4777, 'stiller': 4189, 'reminds': 3620, 'amy': 222, 'fame': 1642, 'cops': 993, 'helps': 2082, 'cox': 1026, 'unusual': 4681, 'cases': 708, 'interpretation': 2328, 'influence': 2286, 'destiny': 1212, 'novels': 3053, 'naked': 2959, 'species': 4098, 'animals': 240, 'hanging': 2023, 'cannibal': 669, 'bo': 521, 'eaten': 1412, 'breasts': 572, 'acts': 107, 'bull': 619, 'rip': 3700, 'rescue': 3643, 'below': 462, 'monkeys': 2891, 'blind': 506, 'contained': 963, 'reed': 3580, 'wars': 4804, 'psychological': 3458, 'suffer': 4275, 'returning': 3667, 'affected': 137, 'military': 2844, 'ready': 3546, '3000': 43, 'invasion': 2340, 'thousands': 4466, 'decide': 1148, 'internet': 2327, 'site': 4005, 'monsters': 2893, 'devoid': 1230, 'catch': 714, 'sleep': 4024, 'board': 522, 'enjoys': 1489, 'mundane': 2938, 'fantastic': 1651, 'reflect': 3584, 'offer': 3087, 'childrens': 793, 'excitement': 1562, 'fights': 1710, 'teaching': 4377, 'available': 361, 'mtv': 2933, 'exactly': 1551, 'scream': 3826, 'titles': 4506, 'deserve': 1199, 'seriously': 3892, '60': 53, 'eighties': 1437, 'digital': 1251, 'sounding': 4083, 'device': 1227, 'drives': 1367, 'stunts': 4246, 'crash': 1033, 'agent': 150, 'confusing': 942, 'rings': 3699, 'todd': 4510, 'assume': 328, 'favor': 1670, 'joy': 2422, 'timeless': 4496, 'aspect': 321, 'irish': 2353, 'covered': 1023, 'incredible': 2269, 'doubts': 1333, 'lily': 2605, 'higher': 2101, 'baby': 376, '1933': 12, 'pre': 3365, 'stanwyck': 4149, 'donald': 1321, 'cook': 988, 'ned': 2986, 'stevens': 4183, 'carter': 703, 'harris': 2038, 'wayne': 4821, 'jimmy': 2403, 'brave': 567, 'therefore': 4439, 'witch': 4909, 'obnoxious': 3067, 'loser': 2655, 'georges': 1894, 'treated': 4573, 'whether': 4866, 'abysmal': 76, 'sitcom': 4004, 'mad': 2696, 'removed': 3624, 'replaced': 3634, 'stupidity': 4248, '50': 51, 'underground': 4640, 'intent': 2318, 'wasting': 4810, 'union': 4661, 'form': 1798, 'epic': 1506, 'negative': 2991, 'restored': 3659, 'literally': 2622, 'steve': 4181, 'comparison': 914, 'kim': 2475, 'beauty': 430, 'captivating': 675, 'hair': 2005, 'yellow': 4981, 'legs': 2566, 'surely': 4309, 'unbearable': 4633, 'chick': 786, 'unforgettable': 4652, 'unique': 4662, 'nasty': 2968, 'watchable': 4812, 'appealing': 272, 'woods': 4935, 'trailer': 4554, 'hollywoods': 2138, 'luck': 2681, 'suitable': 4285, 'shining': 3933, 'thrill': 4471, 'kinds': 2478, 'remake': 3612, 'yesterday': 4983, 'screaming': 3827, 'scary': 3807, 'terribly': 4408, 'noise': 3024, 'directly': 1263, 'robots': 3718, 'notch': 3043, 'decades': 1145, 'spectacular': 4101, 'gary': 1873, 'cooper': 990, 'dozens': 1339, 'classics': 837, 'commercial': 902, 'spielberg': 4110, 'jackson': 2375, 'months': 2897, '70': 55, 'performed': 3230, 'beating': 425, 'assumed': 329, 'issues': 2365, 'summary': 4290, 'speaking': 4095, 'gory': 1943, 'stephen': 4176, 'kings': 2481, 'outstanding': 3147, 'central': 735, 'died': 1244, 'gruesome': 1986, 'mary': 2751, 'visual': 4757, 'genuinely': 1892, 'depiction': 1185, 'zombie': 4997, 'laughable': 2530, 'menacing': 2807, 'creep': 1051, 'massacre': 2754, 'kitchen': 2485, 'table': 4346, 'emotions': 1461, 'stuck': 4237, 'month': 2896, 'train': 4556, 'practically': 3362, 'listening': 2620, 'documentary': 1307, 'guilty': 1995, 'originally': 3131, 'messages': 2820, 'press': 3379, 'impressive': 2253, 'remotely': 3623, 'smooth': 4041, 'powerful': 3360, 'hates': 2048, 'robot': 3717, 'speed': 4103, 'demon': 1177, 'performing': 3233, 'ironically': 2356, 'buddy': 609, 'grace': 1950, 'closed': 854, 'doors': 1329, 'beatty': 427, 'useless': 4695, 'proud': 3444, 'ignorant': 2223, 'hat': 2045, 'rape': 3521, 'warrior': 4802, 'hundreds': 2196, 'feet': 1688, 'lewis': 2586, 'nuts': 3064, 'burt': 629, 'reynolds': 3684, '1972': 18, 'pacing': 3163, 'encounter': 1466, 'symbolism': 4341, 'core': 995, 'blade': 495, 'taught': 4370, 'specific': 4099, 'rules': 3754, 'raw': 3532, 'raise': 3507, 'spirit': 4113, 'subject': 4253, 'interview': 2329, 'disney': 1289, 'broadcast': 591, 'fare': 1655, 'youd': 4987, 'porn': 3336, 'lesbian': 2573, 'brief': 579, 'exotic': 1574, 'cuts': 1091, 'mouth': 2920, 'imagery': 2234, 'monster': 2892, 'youve': 4995, 'logical': 2639, 'element': 1442, 'mediocre': 2791, 'philosophy': 3254, 'provoking': 3452, 'enterprise': 1495, 'drawing': 1350, 'task': 4368, 'notes': 3046, 'philosophical': 3253, 'silence': 3974, 'recorded': 3574, 'cinematic': 818, 'exploration': 1593, 'serving': 3897, 'bomb': 531, '1990': 24, 'theyll': 4444, 'greedy': 1969, 'cold': 872, 'mountain': 2917, 'range': 3516, 'criminals': 1058, 'america': 212, 'produced': 3414, 'worried': 4949, 'lonely': 2643, 'raising': 3509, 'aging': 153, 'easier': 1405, 'united': 4664, 'flying': 1764, 'jet': 2400, 'fighter': 1708, 'questionable': 3488, 'iraq': 2351, 'compare': 911, 'saving': 3797, 'dawn': 1127, 'accurate': 90, 'mini': 2854, 'inspiration': 2300, 'saga': 3773, 'superficial': 4298, 'authentic': 358, 'prepared': 3371, 'mix': 2876, 'hoped': 2156, 'lengthy': 2570, 'generations': 1884, 'growing': 1983, 'eric': 1514, 'finds': 1728, 'center': 732, 'harsh': 2040, 'et': 1526, 'perform': 3227, 'talents': 4356, 'peoples': 3222, 'depicting': 1184, 'admittedly': 123, 'process': 3412, 'somewhere': 4069, 'conclusion': 936, 'status': 4165, 'spends': 4107, 'examples': 1554, 'racist': 3503, 'hence': 2083, 'asked': 317, 'halloween': 2010, 'weekend': 4838, 'ashamed': 312, 'suggest': 4280, 'selling': 3873, 'cat': 713, 'usa': 4691, 'landing': 2512, 'spread': 4133, 'basic': 406, 'conspiracy': 956, 'scientists': 3819, 'lie': 2590, 'shadows': 3913, 'artificial': 306, 'lighting': 2596, 'studio': 4240, 'doubt': 1332, 'outside': 3146, 'basement': 405, 'lies': 2591, 'borrowed': 545, 'weeks': 4839, 'brando': 566, 'lloyd': 2630, 'japan': 2381, 'laws': 2540, 'mike': 2838, 'dancers': 1105, 'san': 3780, 'fancy': 1649, 'compelling': 916, 'relations': 3596, 'hitting': 2126, 'previously': 3388, 'broadway': 592, 'south': 4087, 'staying': 4168, 'source': 4086, 'nation': 2969, 'hey': 2095, 'description': 1197, 'zombies': 4998, 'suck': 4269, 'serial': 3889, 'corpse': 999, 'ocean': 3079, 'sharp': 3922, 'clothes': 860, 'wearing': 4831, 'behave': 445, 'pg': 3250, '13': 4, 'glimpse': 1919, 'exact': 1550, 'topless': 4527, 'trite': 4589, 'station': 4164, 'walls': 4783, 'overly': 3155, 'failing': 1627, 'opposed': 3122, 'korean': 2495, 'indie': 2276, 'par': 3184, 'christian': 808, 'charlotte': 766, 'swedish': 4335, 'expectations': 1576, 'realized': 3553, 'seven': 3903, 'falling': 1639, 'thin': 4449, 'unintentionally': 4659, 'focusing': 1769, 'relationships': 3598, 'lover': 2672, 'gabriel': 1859, 'balls': 390, 'ellen': 1446, 'cameos': 658, 'joey': 2408, 'wreck': 4965, 'fitting': 1742, 'goodness': 1938, 'horribly': 2164, 'bbc': 417, 'worthy': 4956, 'cliche': 843, 'dragged': 1343, 'werent': 4849, 'eccentric': 1414, 'weight': 4840, 'aunt': 355, 'indian': 2274, 'profound': 3423, 'riding': 3695, 'suspects': 4328, 'virtually': 4751, 'unknown': 4668, 'lazy': 2543, 'shocked': 3939, 'disneys': 1290, 'ten': 4399, 'portion': 3338, 'rushed': 3760, 'required': 3641, 'viewing': 4740, 'recording': 3575, 'prior': 3400, 'miracle': 2860, 'walker': 4778, 'mouse': 2919, 'mountains': 2918, 'direct': 1258, 'snow': 4045, 'turner': 4607, 'placed': 3283, 'mildly': 2840, 'soul': 4079, 'gothic': 1945, 'beast': 422, 'roll': 3728, 'web': 4833, 'fu': 1844, 'turkey': 4604, 'suited': 4286, 'kurt': 2499, 'avoided': 364, 'environment': 1505, 'population': 3335, 'survive': 4321, 'destroy': 1213, 'reasonably': 3559, 'corny': 997, 'stinks': 4192, 'comical': 894, 'unintentional': 4658, 'actions': 101, 'routine': 3743, 'pack': 3165, 'quick': 3490, 'combat': 882, 'captured': 677, 'persona': 3239, 'pulls': 3464, 'martial': 2747, 'gratuitous': 1961, 'mechanical': 2788, 'fx': 1858, 'minimal': 2855, 'brosnan': 597, 'intense': 2316, 'contrast': 975, 'davis': 1126, 'gem': 1877, 'pleased': 3305, 'southern': 4088, 'chorus': 803, 'accepted': 82, 'offended': 3085, '11': 2, 'understood': 4647, 'sat': 3786, '40': 48, 'lasted': 2522, 'poem': 3312, 'switch': 4339, 'channels': 754, 'tear': 4379, 'reached': 3537, 'contain': 962, 'warriors': 4803, 'wore': 4939, 'bin': 483, 'handed': 2016, 'irrelevant': 2358, 'avoid': 363, 'nose': 3037, 'makeup': 2717, 'danes': 1108, 'multi': 2935, 'peters': 3248, 'followed': 1773, 'loaded': 2632, 'legend': 2564, 'atlantis': 332, 'maria': 2736, 'ships': 3935, 'judging': 2425, 'creates': 1038, 'explains': 1589, 'breath': 573, 'dances': 1106, 'grade': 1951, 'yelling': 4980, 'remaining': 3610, 'disbelief': 1277, 'creation': 1040, 'junior': 2435, 'fly': 1763, 'drag': 1342, 'reading': 3544, 'pages': 3168, 'stone': 4197, 'sword': 4340, 'tall': 4362, 'steel': 4173, 'luckily': 2682, 'concerns': 934, 'princess': 3397, 'universe': 4666, 'godzilla': 1929, 'stranger': 4213, 'shadow': 3912, 'studios': 4241, 'samurai': 3779, 'mid': 2832, 'rival': 3706, 'shoots': 3944, 'teeth': 4393, 'paper': 3183, 'sold': 4052, 'plane': 3288, 'essentially': 1523, 'someones': 4065, 'pops': 3332, 'worlds': 4947, 'plan': 3287, 'surprising': 4315, 'crush': 1072, 'occasionally': 3075, 'tiger': 4491, 'reaction': 3541, 'stiff': 4187, 'picked': 3266, 'fate': 1665, 'silent': 3975, 'generally': 1881, 'cry': 1073, 'intrigue': 2333, 'dawson': 1128, 'beaten': 424, 'stole': 4194, 'mill': 2845, 'unoriginal': 4673, 'tim': 4494, 'survivor': 4324, 'cell': 730, 'eerie': 1427, 'hook': 2152, 'stock': 4193, 'felix': 1689, 'worn': 4948, 'butt': 638, 'hideous': 2098, 'beneath': 464, 'whoever': 4872, 'popcorn': 3331, 'push': 3477, 'depression': 1189, 'standards': 4145, 'australian': 357, 'dramas': 1347, 'response': 3654, 'situations': 4009, 'individuals': 2278, 'australia': 356, 'occurred': 3077, 'handled': 2019, 'creativity': 1042, 'fred': 1825, 'feminist': 1695, 'wrote': 4974, 'arts': 310, 'loose': 2650, 'buck': 605, 'silver': 3977, 'worthwhile': 4955, 'thankfully': 4422, 'stargate': 4151, 'stilted': 4190, 'russell': 3761, 'dean': 1139, 'anderson': 227, 'asleep': 320, 'smaller': 4034, 'indians': 2275, 'bang': 393, 'explosions': 1597, 'vietnam': 4735, 'key': 2458, 'stories': 4205, 'solely': 4056, 'boot': 538, 'uk': 4626, 'inferior': 2285, 'mexican': 2826, 'fed': 1682, 'continue': 969, 'received': 3564, 'mexico': 2827, 'leslie': 2574, 'howard': 2182, 'mildred': 2841, 'nomination': 3026, 'aimed': 162, 'arguably': 289, 'spin': 4112, 'starred': 4153, 'colonel': 876, 'dr': 1340, 'battle': 414, 'rebel': 3561, 'numerous': 3062, '2002': 32, 'resident': 3647, 'returned': 3666, 'june': 2433, '2003': 33, 'march': 2734, '2005': 35, 'represented': 3638, 'defeat': 1157, 'regardless': 3590, 'cameron': 661, 'mitchell': 2875, 'con': 928, 'artist': 307, 'joined': 2413, 'hank': 2024, 'valuable': 4708, 'franchise': 1817, 'resulting': 3661, '2006': 36, '2007': 37, 'gods': 1928, 'finale': 1723, 'ireland': 2352, 'popularity': 3334, 'animated': 241, 'gain': 1863, 'recognition': 3568, 'relatively': 3600, 'confidence': 938, 'holding': 2130, 'chuck': 812, 'acid': 95, 'requires': 3642, 'defend': 1158, 'lion': 2614, 'matches': 2764, 'mans': 2730, 'navy': 2976, 'eyed': 1612, 'improvement': 2256, 'india': 2273, 'scale': 3802, 'orson': 3132, 'demands': 1175, 'trust': 4598, 'opinions': 3119, 'imdb': 2240, 'polanski': 3321, 'creative': 1041, 'lucky': 2683, 'glenn': 1918, 'achieved': 93, 'altogether': 202, 'visuals': 4759, 'whale': 4857, 'accidentally': 85, 'mark': 2739, 'mate': 2765, 'hurt': 2202, 'motives': 2916, 'grows': 1985, 'progresses': 3426, 'loss': 2659, 'loses': 2657, 'jaws': 2385, 'goal': 1925, 'victim': 4729, 'anywhere': 263, 'pays': 3218, 'entry': 1504, 'wall': 4781, 'accompanied': 86, 'area': 286, 'fat': 1663, 'includes': 2263, 'relevant': 3604, 'equivalent': 1512, 'repeated': 3631, 'aforementioned': 139, 'couples': 1017, 'cringe': 1059, 'inducing': 2279, 'outfit': 3144, 'atrocious': 335, 'blew': 505, 'occasion': 3073, 'writes': 4970, 'marty': 2749, 'misses': 2868, 'island': 2361, 'reunion': 3669, 'wolf': 4920, 'clumsy': 866, 'trio': 4587, 'behavior': 446, 'vote': 4766, 'brazil': 568, 'adam': 111, 'sandler': 3781, 'figured': 1712, 'comedian': 887, 'jesus': 2399, 'christ': 807, 'grasp': 1960, 'importance': 2246, 'intention': 2319, 'critical': 1062, 'text': 4419, 'disagree': 1269, 'insulting': 2310, 'wonderfully': 4929, 'charming': 768, 'assistant': 326, 'waitress': 4773, 'portrays': 3344, 'subsequent': 4257, 'furthermore': 1854, 'scare': 3803, 'vegas': 4717, 'candy': 668, 'strongly': 4231, 'intelligence': 2313, 'rendition': 3625, 'sidney': 3968, 'size': 4012, 'kingdom': 2480, 'obsessed': 3069, 'chasing': 772, 'oliver': 3102, 'potentially': 3356, 'described': 1195, 'fever': 1699, 'goofy': 1939, 'shake': 3914, 'sports': 4130, 'nicholas': 3010, 'lying': 2690, 'suffice': 4279, 'dennis': 1179, 'hopper': 2161, 'flynn': 1765, 'christmas': 810, 'haunted': 2050, 'viewed': 4737, 'temple': 4398, 'mass': 2753, 'pal': 3177, 'till': 4493, 'lit': 2621, 'hong': 2149, 'kong': 2494, 'exposure': 1599, 'ambitious': 211, 'chinese': 796, 'lacks': 2505, 'chest': 784, 'experienced': 1581, 'sacrifice': 3766, 'essential': 1522, 'elvira': 1450, 'regret': 3591, 'classical': 836, 'superbly': 4297, 'rachel': 3500, 'warming': 4795, 'birds': 485, 'vast': 4716, 'ruth': 3763, 'dress': 1357, 'sun': 4292, 'ears': 1402, 'community': 908, 'bud': 607, 'coffee': 870, 'passing': 3204, 'deeply': 1156, 'cars': 702, 'hopeless': 2158, 'despair': 1207, 'learns': 2553, 'absence': 70, 'overcome': 3151, 'effectively': 1430, 'discovery': 1282, 'cup': 1080, 'theaters': 4428, 'repeat': 3630, 'ranks': 3519, 'grave': 1962, 'doom': 1326, 'slapstick': 4019, 'cleverly': 842, 'opposite': 3123, 'comedic': 888, 'barry': 401, 'royal': 3746, 'forms': 1801, 'fantasy': 1652, 'stanley': 4148, 'food': 1778, 'styles': 4250, 'reference': 3582, 'artistic': 308, 'ingredients': 2289, 'extra': 1606, 'sid': 3964, 'interaction': 2322, 'streisand': 4218, 'wishing': 4907, 'multiple': 2936, 'ticket': 4487, 'advanced': 128, 'screening': 3830, 'frustrated': 1842, 'flow': 1762, 'fabulous': 1615, 'magical': 2704, 'pleasantly': 3303, 'deserved': 1200, 'chases': 771, 'duke': 1383, 'pair': 3176, 'simpson': 3987, 'improved': 2255, 'crazy': 1035, 'intentions': 2321, 'testament': 4417, 'endings': 1472, 'idiot': 2218, 'contest': 967, 'equipment': 1511, 'skin': 4015, 'melting': 2799, 'parties': 3196, 'quirky': 3493, 'groups': 1981, 'hated': 2047, 'worthless': 4954, 'dialogs': 1233, 'ford': 1789, 'adds': 119, 'titanic': 4503, 'challenge': 743, 'kate': 2446, 'rose': 3739, 'elderly': 1440, 'emotionally': 1460, 'lifetime': 2594, 'eg': 1434, 'freeman': 1829, 'react': 3540, 'lousy': 2667, 'sincere': 3991, 'pitch': 3279, 'dickens': 1240, 'edward': 1426, 'storm': 4206, 'travels': 4569, 'mummy': 2937, 'cure': 1081, 'chaos': 755, 'affect': 136, 'steps': 4177, 'gonna': 1936, 'details': 1219, 'arms': 294, 'thoughts': 4464, 'lynch': 2691, 'exceptional': 1558, 'sinister': 3998, 'traditional': 4550, 'painted': 3174, 'dressing': 1359, 'heights': 2073, 'inevitably': 2283, 'ultimate': 4627, 'stylish': 4251, 'tongue': 4518, 'tied': 4489, 'fully': 1847, 'bourne': 554, 'caused': 724, 'trademark': 4548, 'training': 4558, 'trick': 4582, 'crucial': 1068, 'prequel': 3372, 'understandable': 4644, 'damon': 1101, 'trade': 4547, 'equal': 1509, 'alive': 186, 'circle': 821, 'report': 3635, 'vengeance': 4719, 'pamela': 3179, 'tomatoes': 4515, 'eating': 1413, 'superman': 4301, 'pretend': 3381, 'print': 3399, 'mistress': 2874, 'nightmare': 3014, 'overdone': 3152, 'twisted': 4618, 'lights': 2597, 'toys': 4543, 'discussion': 1284, 'amazingly': 210, 'covers': 1024, 'empty': 1465, 'matters': 2770, 'dig': 1250, 'ourselves': 3140, 'helicopter': 2076, 'twelve': 4611, 'troubled': 4593, 'attorney': 348, 'warmth': 4796, 'swim': 4337, 'bold': 528, 'innovative': 2295, 'genius': 1887, 'formula': 1802, 'opportunities': 3120, 'dreary': 1356, 'iron': 2354, 'sheer': 3925, 'originality': 3130, 'highlight': 2103, 'depressed': 1187, 'stewart': 4184, 'secretly': 3851, 'letters': 2582, 'whilst': 4869, 'morgan': 2905, 'arrogant': 303, 'mail': 2706, 'suspect': 4327, 'rocket': 3721, 'explore': 1594, 'mars': 2746, 'heavy': 2070, 'consists': 955, 'bride': 577, 'winds': 4895, 'flesh': 1753, 'wood': 4933, 'psychiatrist': 3455, 'virus': 4752, 'survived': 4322, 'send': 3875, 'jeffrey': 2391, 'kidnapped': 2466, 'precious': 3366, 'paltrow': 3178, 'capturing': 679, 'attitude': 346, 'emma': 1457, 'elegant': 1441, 'reduced': 3579, 'scientist': 3818, 'independent': 2272, 'wonders': 4931, 'struggles': 4235, 'denzel': 1181, 'shortly': 3947, 'cards': 683, 'doc': 1303, 'associated': 327, 'guessed': 1990, 'hearts': 2066, 'specifically': 4100, 'satan': 3787, 'presented': 3376, 'nowhere': 3056, 'cardboard': 682, 'forbidden': 1785, 'satire': 3788, 'majority': 2712, 'nail': 2957, 'claims': 830, 'corporate': 998, 'media': 2789, 'control': 977, 'outrageous': 3145, 'poetic': 3313, 'trilogy': 4586, 'boat': 523, 'resist': 3648, 'cake': 648, 'successfully': 4267, 'minded': 2850, 'lay': 2542, 'tiny': 4500, 'graphic': 1958, 'anime': 243, 'ad': 110, 'matrix': 2767, 'punk': 3468, 'bands': 392, 'reaches': 3538, 'france': 1816, 'pride': 3391, 'honor': 2150, 'tremendous': 4579, 'expert': 1585, 'justify': 2439, '1st': 28, 'robin': 3715, 'landscapes': 2514, 'nonsensical': 3031, 'exercise': 1568, 'meaningless': 2782, 'witches': 4910, 'wells': 4845, 'somebody': 4062, 'belong': 459, 'doll': 1313, 'resembles': 3646, 'sucked': 4270, 'believed': 455, 'batman': 413, 'irony': 2357, 'grab': 1949, 'norman': 3035, 'buff': 611, 'angel': 231, 'richards': 3687, 'blatant': 502, 'mask': 2752, 'disturbing': 1300, 'alice': 182, 'uncomfortable': 4637, 'ridiculously': 3694, 'rabbit': 3498, 'imitation': 2241, 'kicking': 2462, 'gifted': 1907, 'noted': 3045, 'karloff': 2445, 'religion': 3607, 'figures': 1713, 'happily': 2030, 'sisters': 4002, 'losing': 2658, 'trap': 4563, 'ritter': 3705, 'eva': 1531, 'lake': 2509, 'strictly': 4223, 'jason': 2383, 'kicked': 2461, 'traveling': 4568, 'contact': 961, 'ashley': 313, 'teach': 4374, 'thrilling': 4474, 'hooked': 2153, 'friendship': 1838, 'colour': 880, 'convinced': 984, 'babe': 375, 'vs': 4767, 'flop': 1760, 'performer': 3231, 'females': 1694, 'pass': 3201, 'players': 3299, 'unaware': 4632, 'sexuality': 3909, 'doctors': 1305, 'urban': 4688, 'chilling': 794, 'inventive': 2341, 'wholly': 4874, 'ideal': 2214, 'spirited': 4114, 'blake': 498, 'scottish': 3825, 'rob': 3710, 'battles': 415, 'rough': 3741, 'purchase': 3469, 'sell': 3871, 'improve': 2254, 'noble': 3021, 'responsibility': 3655, 'rotten': 3740, 'steals': 4172, 'settle': 3902, 'holds': 2131, 'dignity': 1252, 'sleazy': 4023, 'hop': 2154, 'astaire': 330, 'subplot': 4255, 'ginger': 1908, 'ball': 388, 'partner': 3198, 'sings': 3997, 'poster': 3353, 'engaged': 1480, 'freak': 1824, 'everyday': 1541, 'buffs': 612, 'hammer': 2014, 'reviewer': 3677, 'pushed': 3478, 'audio': 354, 'pointed': 3317, 'scares': 3806, 'existent': 1572, 'souls': 4080, 'design': 1202, 'stress': 4220, 'cabin': 643, 'newspaper': 3006, 'neat': 2982, 'worry': 4950, 'dust': 1388, 'thick': 4447, 'cable': 644, 'toward': 4539, 'shines': 3932, 'daring': 1114, 'cia': 814, 'sequels': 3886, 'notice': 3048, 'superhero': 4299, 'unit': 4663, 'closet': 858, 'dealt': 1138, 'closer': 856, 'neighbor': 2992, 'basketball': 409, 'creator': 1043, 'shorts': 3948, 'abusive': 75, 'breathtaking': 574, 'deaths': 1142, 'factor': 1622, 'hbo': 2055, 'rental': 3627, 'bay': 416, 'hippie': 2115, 'conservative': 948, 'interviews': 2330, 'reel': 3581, 'smoke': 4039, 'madness': 2698, 'nights': 3016, '18': 10, 'alright': 198, 'met': 2822, 'relies': 3606, 'draw': 1349, 'achieve': 92, 'theyve': 4446, 'court': 1020, 'bears': 421, 'relation': 3595, 'spring': 4134, 'catching': 716, 'guard': 1988, 'meaningful': 2781, 'urge': 4689, 'overwhelming': 3157, 'secondly': 3847, 'motivation': 2914, 'enthusiasm': 1501, 'exposed': 1598, 'nevertheless': 3002, 'consistently': 954, 'movements': 2924, 'ought': 3138, 'principal': 3398, 'realizing': 3555, 'burn': 625, 'drunk': 1375, 'highest': 2102, 'steven': 4182, 'credibility': 1047, 'tour': 4538, 'virginia': 4750, 'narrator': 2967, 'burton': 630, 'experiments': 1584, 'mission': 2870, 'test': 4416, 'dollar': 1314, 'plastic': 3294, 'thirty': 4456, 'rap': 3520, 'immensely': 2243, 'essence': 1521, 'massive': 2756, '1968': 16, 'sole': 4055, 'ironic': 2355, 'leo': 2571, 'allowing': 191, '2001': 31, 'heston': 2094, 'command': 897, 'charisma': 762, 'icon': 2211, 'prison': 3401, 'surface': 4310, 'courage': 1018, 'spots': 4132, 'support': 4303, 'hed': 2072, 'repeatedly': 3632, 'hire': 2116, 'whatsoever': 4861, '2000': 30, 'dreams': 1355, 'model': 2880, 'laura': 2536, '1999': 27, 'park': 3190, 'persons': 3244, 'suffered': 4276, 'chicago': 785, 'projects': 3428, '1970s': 17, 'struck': 4232, 'page': 3167, 'fields': 1705, 'craft': 1028, 'serve': 3893, 'masses': 2755, 'alfred': 181, 'wwii': 4975, 'buildings': 616, 'miserably': 2864, 'itll': 2369, '13th': 5, 'birthday': 487, 'dorothy': 1330, 'pat': 3208, 'unbelievably': 4635, 'iran': 2350, 'native': 2972, 'primarily': 3393, 'education': 1424, 'learning': 2552, 'remains': 3611, 'author': 359, 'claire': 831, 'isolated': 2363, 'base': 402, 'commentary': 899, 'alternate': 200, 'factory': 1623, 'jerk': 2395, 'hello': 2078, 'simplicity': 3984, 'pan': 3180, 'artists': 309, 'express': 1600, 'appearances': 275, 'possibility': 3349, 'resemblance': 3645, 'dragon': 1344, 'calm': 655, 'machines': 2694, 'term': 4405, 'amongst': 218, 'clichéd': 845, 'shed': 3924, 'bare': 398, 'campy': 664, 'teachers': 4376, 'pseudo': 3454, 'awfully': 373, 'learned': 2551, 'practice': 3363, 'carol': 693, 'barbara': 397, 'failure': 1629, 'cares': 690, 'user': 4696, 'jesse': 2397, 'faster': 1662, 'caine': 647, 'championship': 745, 'mansion': 2731, 'prom': 3429, 'hamilton': 2012, 'ladies': 2506, 'watches': 4814, 'domestic': 1318, 'muslim': 2949, 'legal': 2563, 'involve': 2345, 'wives': 4918, 'seeking': 3859, 'divorce': 1301, 'investigation': 2343, 'receive': 3563, 'justice': 2438, 'gender': 1878, 'benefit': 465, 'victoria': 4732, 'striking': 4226, 'emily': 1456, 'blunt': 520, 'rise': 3702, 'marriage': 2743, 'chess': 783, 'territory': 4411, 'attached': 336, 'civilization': 827, 'health': 2060, 'occur': 3076, 'desperation': 1210, 'field': 1704, 'intimate': 2331, 'succeeds': 4264, 'offered': 3088, 'directions': 1262, '1950s': 14, 'musicals': 2948, 'powell': 3358, 'robinson': 3716, 'ruby': 3749, 'shirley': 3936, 'threatening': 4468, 'similarities': 3979, 'julie': 2428, 'andrews': 229, 'anna': 245, 'bite': 489, 'cowboy': 1025, 'westerns': 4854, 'shape': 3919, 'scooby': 3820, 'doo': 1325, '1990s': 25, 'tune': 4602, 'louis': 2665, 'catchy': 717, 'merits': 2817, 'basis': 408, 'resolution': 3649, 'regard': 3588, 'timing': 4498, 'program': 3424, 'weapons': 4829, 'sits': 4006, 'susan': 4326, 'severe': 3906, 'mall': 2720, 'weapon': 4828, 'distance': 1294, 'hood': 2151, 'fix': 1744, 'tribute': 4581, 'dude': 1381, 'perspective': 3245, 'possibilities': 3348, 'reflection': 3585, 'wilson': 4891, 'bed': 436, 'intrigued': 2334, 'terrorist': 4413, 'civil': 826, 'considerable': 950, 'staff': 4138, 'misery': 2865, 'moral': 2902, 'beach': 419, 'arrive': 300, 'confusion': 943, 'introduced': 2337, 'lawrence': 2539, 'ease': 1404, 'subjects': 4254, 'medium': 2792, 'cinemas': 817, 'damage': 1099, 'position': 3345, 'europe': 1529, 'stereotype': 4178, 'tap': 4364, 'priceless': 3390, 'liners': 2611, 'fallen': 1638, 'blend': 504, 'curse': 1086, 'partly': 3197, 'costume': 1005, 'dressed': 1358, 'gotta': 1946, 'instantly': 2306, 'brutally': 604, 'proceedings': 3410, 'adapted': 114, 'scope': 3821, 'guilt': 1994, 'chan': 746, 'path': 3209, 'solution': 4059, 'tender': 4401, 'fashion': 1659, 'heck': 2071, 'nazis': 2978, 'hollow': 2135, 'kicks': 2463, 'primary': 3394, 'extended': 1604, 'abandoned': 62, 'bat': 410, 'stooges': 4200, 'horses': 2171, 'darkness': 1117, 'cave': 727, 'strip': 4228, 'handful': 2017, 'sappy': 3784, 'gangsters': 1869, 'clark': 832, 'duo': 1386, 'catches': 715, 'careers': 687, 'nurse': 3063, 'brooks': 596, 'scores': 3823, 'strangely': 4212, 'plague': 3285, 'mothers': 2912, 'gas': 1874, 'officers': 3093, 'loosely': 2651, 'exploitation': 1592, 'photographer': 3259, 'moody': 2899, 'directs': 1267, 'michelle': 2830, 'ghosts': 1903, 'uwe': 4703, 'boll': 529, 'pile': 3275, 'humble': 2191, 'overrated': 3156, 'quit': 3494, 'bringing': 586, 'centered': 733, 'games': 1865, 'garbo': 1871, 'cinderella': 815, 'showcase': 3954, 'appreciation': 281, 'drew': 1360, 'inspiring': 2302, 'mickey': 2831, 'beloved': 461, 'represent': 3637, 'admire': 121, 'tonight': 4519, 'plausible': 3295, 'homage': 2141, 'composed': 926, 'propaganda': 3435, 'enormous': 1490, 'schools': 3814, 'sidekick': 3966, 'nostalgic': 3039, 'returns': 3668, 'lou': 2663, 'scrooge': 3837, 'holiday': 2134, 'poignant': 3315, 'dolph': 1317, 'nightmares': 3015, 'unrealistic': 4677, 'cheating': 776, 'womans': 4922, 'nominated': 3025, 'illogical': 2231, 'decisions': 1152, 'gandhi': 1866, 'describes': 1196, 'freedom': 1828, 'dialogues': 1235, 'bobby': 525, 'disappoint': 1272, 'outcome': 3142, 'mindless': 2851, 'julia': 2427, 'relatives': 3601, 'displays': 1293, 'mistaken': 2872, 'sheriff': 3929, 'escapes': 1519, 'celluloid': 731, 'spending': 4106, 'knife': 2487, 'liberal': 2588, 'wrapped': 4964, 'spiritual': 4116, 'niro': 3019, 'gold': 1932, 'desired': 1206, 'solve': 4060, 'synopsis': 4344, '35': 45, 'bush': 632, 'supernatural': 4302, 'lucas': 2680, 'thoughtful': 4463, 'commit': 904, 'advantage': 129, 'existence': 1571, 'soldier': 4053, 'waves': 4819, 'trashy': 4566, 'dear': 1140, 'sends': 3876, 'treats': 4575, 'christians': 809, 'italy': 2368, 'instance': 2304, 'marketing': 2741, 'fulci': 1845, 'presumably': 3380, 'swimming': 4338, 'notably': 3042, 'recall': 3562, 'molly': 2884, 'props': 3439, 'laughably': 2531, 'commented': 900, 'travesty': 4570, 'warner': 4799, 'format': 1799, 'ron': 3734, 'occasional': 3074, 'minimum': 2856, 'slaughter': 4021, 'throat': 4476, 'vicious': 4728, '75': 57, 'wealth': 4826, 'glory': 1922, 'boom': 537, 'carrey': 696, 'homeless': 2143, 'suggests': 4282, 'ignored': 2225, 'acceptable': 81, 'ties': 4490, 'marry': 2745, 'toilet': 4512, 'monkey': 2890, 'spike': 4111, 'cameo': 657, 'purposes': 3475, 'favorites': 1672, 'careful': 688, 'pie': 3271, 'kirk': 2482, 'contract': 973, '3rd': 47, 'chair': 742, 'safety': 3772, 'challenging': 744, 'credible': 1048, 'letter': 2581, 'charlie': 765, 'halfway': 2008, 'lesser': 2576, 'hal': 2006, 'property': 3438, 'appalling': 268, 'murphy': 2944, 'worker': 4942, 'demons': 1178, 'load': 2631, 'lab': 2501, 'minds': 2852, 'fish': 1738, 'frankenstein': 1822, 'miller': 2846, 'notion': 3050, 'generic': 1885, 'sadistic': 3768, 'unwatchable': 4682, 'roberts': 3714, 'comics': 895, 'airplane': 166, 'flies': 1756, 'matthew': 2772, 'carrying': 701, 'unsettling': 4679, 'companion': 909, 'fears': 1677, 'enjoyment': 1488, 'california': 650, 'vincent': 4746, 'showdown': 3955, 'walks': 4780, 'troubles': 4594, 'da': 1094, 'ramones': 3512, 'clint': 850, 'agreed': 156, 'stones': 4198, 'catherine': 719, 'continued': 970, 'instinct': 2308, 'agrees': 157, 'technically': 4382, 'overlook': 3153, 'sympathy': 4343, 'subplots': 4256, 'attempting': 342, 'sexually': 3910, 'flashback': 1746, 'dire': 1257, 'reaching': 3539, 'university': 4667, 'blacks': 494, 'cube': 1076, 'survival': 4320, 'expressed': 1601, 'gradually': 1952, 'proceeds': 3411, 'link': 2613, 'midnight': 2834, 'hoffman': 2128, 'identify': 2216, 'importantly': 2248, 'forty': 1807, 'morris': 2908, 'bird': 484, 'richardson': 3688, 'nervous': 2999, 'crimes': 1056, 'transformation': 4560, 'chicks': 787, 'elizabeth': 1445, 'draws': 1352, 'der': 1191, 'grandfather': 1954, 'hamlet': 2013, 'shakespeare': 3915, 'thumbs': 4485, 'hype': 2207, 'neck': 2985, 'sleeping': 4025, 'row': 3744, 'bela': 450, 'lugosi': 2686, 'demented': 1176, 'charismatic': 763, 'witnessed': 4915, 'eyre': 1614, 'dalton': 1098, 'honesty': 2148, 'virgin': 4749, 'khan': 2459, 'larger': 2519, 'seeks': 3860, 'bollywood': 530, 'kapoor': 2443, 'promises': 3432, 'intentionally': 2320, 'stronger': 4230, 'alcohol': 176, 'critics': 1064, 'cheese': 780, 'eve': 1532, 'brutal': 603, 'brenda': 575, 'warm': 4794, 'morality': 2903, 'choreography': 802, 'frequently': 1832, 'overlooked': 3154, 'cheek': 779, 'preview': 3386, 'jumped': 2430, 'cutting': 1092, 'defined': 1160, 'map': 2733, 'haunting': 2051, 'incomprehensible': 2267, 'bond': 532, 'tunes': 4603, 'vice': 4727, 'charge': 761, 'trial': 4580, '2nd': 41, 'greed': 1968, 'beings': 449, 'maggie': 2702, 'reasonable': 3558, '24': 39, 'investigate': 2342, 'mile': 2842, 'giallo': 1904, 'inspector': 2299, 'phantom': 3251, 'vaguely': 4706, 'wrap': 4963, 'businessman': 634, 'hills': 2109, 'twilight': 4614, 'bergman': 466, 'illness': 2230, 'cried': 1054, 'per': 3223, 'ethan': 1528, 'reactions': 3542, 'destroyed': 1214, 'germans': 1897, 'montage': 2894, 'spell': 4104, 'maintain': 2710, 'womens': 4924, 'wounded': 4961, 'army': 295, 'streep': 4215, 'everywhere': 1545, 'carl': 692, 'finger': 1731, 'bargain': 400, 'illegal': 2229, 'gross': 1978, 'slick': 4026, 'segment': 3867, 'ballet': 389, 'outer': 3143, 'cg': 739, 'generated': 1882, 'screens': 3832, 'carpenter': 694, 'jamie': 2379, 'curtis': 1087, 'myers': 2952, 'nearby': 2980, 'drops': 1372, 'burning': 627, 'wizard': 4919, 'oz': 3160, 'paulie': 3215, 'stretch': 4221, 'cheated': 775, 'threat': 4467, 'develops': 1226, 'coherent': 871, 'hints': 2113, 'bottle': 550, 'murderer': 2941, 'laid': 2508, 'redemption': 3578, '1980': 20, 'foster': 1809, 'riveting': 3708, 'anyways': 262, 'zone': 4999, 'vague': 4705, 'triumph': 4590, 'accomplished': 87, 'afford': 138, 'storytelling': 4209, 'everyones': 1543, 'diamond': 1236, 'kudos': 2497, 'dentist': 1180, 'nelson': 2997, 'menace': 2806, 'raymond': 3534, 'arrival': 299, 'troops': 4591, 'tree': 4576, 'passes': 3203, 'adams': 112, 'blown': 516, 'guessing': 1991, 'francisco': 1819, 'airport': 167, 'toy': 4542, 'risk': 3704, 'gerard': 1895, 'suggested': 4281, 'prostitute': 3440, 'pro': 3406, 'models': 2881, 'descent': 1193, 'strike': 4224, 'liking': 2604, 'branagh': 564, 'tommy': 4516, 'likewise': 2603, 'comparing': 913, 'angela': 232, 'retired': 3664, 'stops': 4203, 'hatred': 2049, 'ruins': 3752, 'bet': 470, 'sullivan': 4288, 'seagal': 3839, 'alcoholic': 177, 'jail': 2376, 'donna': 1323, 'sutherland': 4333, 'rambo': 3511, 'horrors': 2169, 'spain': 4091, 'sandra': 3782, 'moore': 2901, 'embarrassment': 1455, 'peace': 3219, 'conflicts': 940, 'harry': 2039, 'solo': 4058, 'jazz': 2387, 'piano': 3264, 'matched': 2763, 'april': 284, 'resort': 3650, 'namely': 2962, 'elm': 1447, 'achievement': 94, 'terry': 4415, 'rat': 3525, 'bedroom': 437, 'repetitive': 3633, 'waited': 4771, 'sum': 4289, 'dynamic': 1394, 'beer': 439, 'sides': 3967, '1940s': 13, '1960s': 15, 'choreographed': 801, 'sue': 4274, 'thief': 4448, 'dinner': 1254, 'tortured': 4530, 'countryside': 1015, 'dvds': 1392, 'moronic': 2907, 'photos': 3261, 'corrupt': 1001, 'consistent': 953, 'homer': 2144, 'laurel': 2537, 'hardy': 2037, 'stan': 4142, 'rolled': 3729, 'strangers': 4214, 'clown': 862, 'baker': 385, 'futuristic': 1857, 'household': 2179, 'louise': 2666, '45': 50, 'satisfy': 3790, 'cats': 721, 'shelley': 3928, 'horse': 2170, 'bath': 411, 'communist': 907, 'hurts': 2203, 'hopes': 2159, 'unseen': 4678, 'loyal': 2679, 'sally': 3776, 'distracting': 1297, 'conceived': 929, 'rex': 3683, 'innocence': 2293, 'bible': 477, 'butler': 637, 'frequent': 1831, 'wallace': 4782, 'tooth': 4524, 'stretched': 4222, 'oddly': 3081, 'dedicated': 1153, 'belongs': 460, 'secrets': 3852, 'ward': 4793, 'forgettable': 1794, 'montana': 2895, 'eastwood': 1409, 'rocky': 3723, 'bleak': 503, 'influenced': 2287, 'blob': 507, 'cartoons': 705, 'luke': 2687, 'sinatra': 3989, 'arm': 292, 'nancy': 2964, 'introduces': 2338, 'fury': 1855, 'climactic': 848, 'splatter': 4118, 'boxing': 556, 'stunt': 4245, 'masters': 2761, 'bumbling': 622, 'ripped': 3701, 'tea': 4373, 'bonus': 534, 'murderous': 2942, 'wandering': 4785, 'messed': 2821, 'saves': 3796, 'uneven': 4649, 'inevitable': 2282, 'unpredictable': 4675, 'crafted': 1029, 'convincingly': 986, 'sentence': 3881, 'combine': 884, 'cd': 728, 'unreal': 4676, 'bathroom': 412, 'antics': 255, 'delivering': 1171, 'sellers': 3872, 'shower': 3957, 'lisa': 2616, 'snake': 4042, 'installment': 2303, 'robbery': 3711, 'photo': 3257, 'lane': 2515, 'inappropriate': 2259, 'rochester': 3719, 'lively': 2627, 'shy': 3962, 'rank': 3518, 'exaggerated': 1552, 'methods': 2825, 'backgrounds': 381, 'errors': 1516, 'orders': 3127, 'packed': 3166, 'patients': 3212, 'crocodile': 1065, 'mst3k': 2932, 'racial': 3501, 'fond': 1776, 'aids': 161, 'emphasis': 1463, 'lucy': 2684, 'che': 773, 'diana': 1237, 'wed': 4835, 'global': 1920, 'advance': 127, 'providing': 3451, 'disturbed': 1299, 'excessive': 1560, 'depicts': 1186, 'versus': 4722, 'spoke': 4125, 'farm': 1656, 'sixties': 4011, 'lust': 2689, 'wrestling': 4966, 'drunken': 1376, 'beats': 426, 'bettie': 473, 'surviving': 4323, 'imagined': 2239, 'incompetent': 2266, 'cinematographer': 819, 'los': 2653, 'angeles': 233, 'curiosity': 1082, 'pearl': 3220, 'modesty': 2883, 'introduce': 2336, 'undoubtedly': 4648, 'farce': 1654, 'bound': 553, 'hardcore': 2034, 'patrick': 3213, 'gundam': 1997, 'cliff': 847, 'jackie': 2374, 'rick': 3689, 'genres': 1889, 'explored': 1595, 'transfer': 4559, 'amitabh': 216, 'rats': 3531, 'carrie': 697, 'andrew': 228, 'instant': 2305, 'dolls': 1316, 'resources': 3651, 'assigned': 325, 'subtlety': 4261, 'broad': 590, 'possessed': 3347, 'lyrics': 2692, 'kennedy': 2454, 'chooses': 800, 'widmark': 4882, 'pursuit': 3476, 'disease': 1285, 'rage': 3505, 'bullets': 621, 'davies': 1125, 'chaplin': 756, 'derek': 1192, 'soccer': 4048, 'signed': 3971, 'deliberately': 1166, 'centers': 734, 'merit': 2816, 'widely': 4881, 'thru': 4483, 'phony': 3256, 'abc': 63, 'offering': 3089, 'jake': 2377, 'neo': 2998, 'loads': 2633, 'ya': 4976, 'lemmon': 2567, 'cameras': 660, 'kubrick': 2496, 'impress': 2250, 'scarecrow': 3804, 'fisher': 1739, 'snl': 4044, 'daddy': 1096, 'mann': 2728, 'buddies': 608, 'dracula': 1341, 'surfing': 4311, 'abilities': 64, 'assault': 324, 'caliber': 649, 'falk': 1636, 'definite': 1161, 'wes': 4851, 'distribution': 1298, 'september': 3884, 'wicked': 4879, 'drags': 1345, 'holly': 2136, 'shirt': 3937, 'skills': 4014, 'hysterical': 2208, 'akshay': 169, 'miike': 2837, 'signs': 3973, 'edie': 1419, 'shelf': 3926, 'underlying': 4641, 'waters': 4817, 'card': 681, 'gadget': 1860, 'warren': 4801, 'lips': 2615, 'porno': 3337, 'lincoln': 2608, 'abraham': 69, 'plight': 3308, 'columbo': 881, 'pretending': 3382, 'hilariously': 2107, 'domino': 1319, 'argue': 290, 'wendy': 4846, 'spirits': 4115, 'kay': 2447, 'adorable': 124, 'bacall': 377, 'coach': 868, 'stinker': 4191, 'freddy': 1826, 'brady': 561, 'campbell': 663, 'sport': 4129, 'elvis': 1451, 'theyd': 4443, 'winters': 4901, 'weakest': 4825, 'simmons': 3981, 'critic': 1061, 'bakshi': 386, 'kenneth': 2455, 'kissing': 2484, 'deaf': 1134, 'lumet': 2688, 'celebrity': 729, 'craven': 1034, 'feed': 1683, 'online': 3109, '3d': 46, 'lena': 2568, 'hyde': 2206, 'li': 2587}\n"
     ]
    }
   ],
   "source": [
    "print(str(vocabulary))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Once you have added the endpoint name to the Lambda function, click on **Save**. Your Lambda function is now up and running. Next we need to create a way for our web app to execute the Lambda function."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Setting up API Gateway\n",
    "\n",
    "Now that our Lambda function is set up, it is time to create a new API using API Gateway that will trigger the Lambda function we have just created.\n",
    "\n",
    "Using AWS Console, navigate to **Amazon API Gateway** and then click on **Get started**.\n",
    "\n",
    "On the next page, make sure that **New API** is selected and give the new api a name, for example, `sentiment_analysis_web_app`. Then, click on **Create API**.\n",
    "\n",
    "Now we have created an API, however it doesn't currently do anything. What we want it to do is to trigger the Lambda function that we created earlier.\n",
    "\n",
    "Select the **Actions** dropdown menu and click **Create Method**. A new blank method will be created, select its dropdown menu and select **POST**, then click on the check mark beside it.\n",
    "\n",
    "For the integration point, make sure that **Lambda Function** is selected and click on the **Use Lambda Proxy integration**. This option makes sure that the data that is sent to the API is then sent directly to the Lambda function with no processing. It also means that the return value must be a proper response object as it will also not be processed by API Gateway.\n",
    "\n",
    "Type the name of the Lambda function you created earlier into the **Lambda Function** text entry box and then click on **Save**. Click on **OK** in the pop-up box that then appears, giving permission to API Gateway to invoke the Lambda function you created.\n",
    "\n",
    "The last step in creating the API Gateway is to select the **Actions** dropdown and click on **Deploy API**. You will need to create a new Deployment stage and name it anything you like, for example `prod`.\n",
    "\n",
    "You have now successfully set up a public API to access your SageMaker model. Make sure to copy or write down the URL provided to invoke your newly created public API as this will be needed in the next step. This URL can be found at the top of the page, highlighted in blue next to the text **Invoke URL**."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Step 7: Deploying our web app\n",
    "\n",
    "Now that we have a publicly available API, we can start using it in a web app. For our purposes, we have provided a simple static html file which can make use of the public api you created earlier.\n",
    "\n",
    "In the `website` folder there should be a file called `index.html`. Download the file to your computer and open that file up in a text editor of your choice. There should be a line which contains **\\*\\*REPLACE WITH PUBLIC API URL\\*\\***. Replace this string with the url that you wrote down in the last step and then save the file.\n",
    "\n",
    "Now, if you open `index.html` on your local computer, your browser will behave as a local web server and you can use the provided site to interact with your SageMaker model.\n",
    "\n",
    "If you'd like to go further, you can host this html file anywhere you'd like, for example using github or hosting a static site on Amazon's S3. Once you have done this you can share the link with anyone you'd like and have them play with it too!\n",
    "\n",
    "> **Important Note** In order for the web app to communicate with the SageMaker endpoint, the endpoint has to actually be deployed and running. This means that you are paying for it. Make sure that the endpoint is running when you want to use the web app but that you shut it down when you don't need it, otherwise you will end up with a surprisingly large AWS bill."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Delete the endpoint\n",
    "\n",
    "Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 50,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "INFO:sagemaker:Deleting endpoint with name: xgboost-2019-01-14-18-26-14-053\n"
     ]
    }
   ],
   "source": [
    "xgb_predictor.delete_endpoint()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Optional: Clean up\n",
    "\n",
    "The default notebook instance on SageMaker doesn't have a lot of excess disk space available. As you continue to complete and execute notebooks you will eventually fill up this disk space, leading to errors which can be difficult to diagnose. Once you are completely finished using a notebook it is a good idea to remove the files that you created along the way. Of course, you can do this from the terminal or from the notebook hub if you would like. The cell below contains some commands to clean up the created files from within the notebook."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 51,
   "metadata": {},
   "outputs": [],
   "source": [
    "# First we will remove all of the files contained in the data_dir directory\n",
    "!rm $data_dir/*\n",
    "\n",
    "# And then we delete the directory itself\n",
    "!rmdir $data_dir\n",
    "\n",
    "# Similarly we remove the files in the cache_dir directory and the directory itself\n",
    "!rm $cache_dir/*\n",
    "!rmdir $cache_dir"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "conda_python3",
   "language": "python",
   "name": "conda_python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Sentiment Analysis Web App\n',
    '\n',
    '_Deep Learning Nanodegree Program | Deployment_\n',
    '\n',
    '---\n',
    '\n',
    "In this notebook we will use Amazon's SageMaker service to construct a random tree model to predict the sentiment of a movie review. In addition, we will deploy this model to an endpoint and construct a very simple web app which will interact with our model's deployed endpoint.\n",
    '\n',
    '## General Outline\n',
    '\n',
    'Typically, when using a notebook instance with SageMaker, you will proceed through the following steps. Of course, not every step will need to be done with each project. Also, there is quite a lot of room for variation in many of the steps, as you will see throughout these lessons.\n',
    '\n',
    '1. Download or otherwise retrieve the data.\n',
    '2. Process / Prepare the data.\n',
    '3. Upload the processed data to S3.\n',
    '4. Train a chos